## Setting + Result

In [ ]:
!pip install -q openai
import openai, time
import pandas as pd

!rm -f data
!ln -s '<path_to_data_folder>' data

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.1 MB/s eta 0:00:00


In [ ]:
openai.api_key = ""
predictions = []

In [ ]:
def chatgpt(prompt, max_tokens=5):
    while True:
        try:
            return openai.ChatCompletion.create(
                model="gpt-3.5-turbo-0301",
                messages=[{"role": "user", "content": prompt}],
                max_tokens=max_tokens, temperature=0
            )['choices'][0]['message']['content'].strip()
        except Exception as e:
            print(e)
            time.sleep(timeout_failure)

def davinci(prompt, max_tokens=5):
    while True:
        try:
            return openai.Completion.create(
                model="gpt-3.5-turbo-instruct", #"text-davinci-003",
                prompt=[prompt],
                max_tokens=max_tokens, temperature=0
            )['choices'][0]['text'].strip()
        except Exception as e:
            print(e)
            time.sleep(timeout_failure)

In [ ]:
# Tq's modified implementation
import asyncio
import time

async def dispatch_openai_requests(
    messages_list,
    model="gpt-3.5-turbo-0301",
    max_tokens=4,
    ):
    """Dispatches requests to OpenAI API asynchronously."""
    async_responses = [
        openai.ChatCompletion.acreate(
            model=model,
            messages=x,
            temperature=0,
            max_tokens=max_tokens,
            top_p=1,
        )
        for x in messages_list
    ]
    time.sleep(1)
    return await asyncio.gather(*async_responses)


In [ ]:
data_df = pd.read_csv('data/dev_data_to_tune.csv')
data = data_df['assertion'].tolist()

In [ ]:
!hostname -I

172.28.0.12 


## Neubig's code

In [ ]:
import asyncio
from typing import Any

async def dispatch_openai_requests(
    messages_list: list[list[dict[str,Any]]],
    model: str,
    temperature: float,
    max_tokens: int,
    top_p: float,
) -> list[str]:
    """Dispatches requests to OpenAI API asynchronously.

    Args:
        messages_list: List of messages to be sent to OpenAI ChatCompletion API.
        model: OpenAI model to use.
        temperature: Temperature to use for the model.
        max_tokens: Maximum number of tokens to generate.
        top_p: Top p to use for the model.

    Returns:
        List of responses from OpenAI API.
    """
    async_responses = [
        openai.ChatCompletion.acreate(
            model=model,
            messages=x,
            temperature=temperature,
            max_tokens=max_tokens,
            top_p=top_p,
        )
        for x in messages_list
    ]
    return await asyncio.gather(*async_responses)


In [ ]:
predictions = asyncio.run(
    dispatch_openai_requests(
        messages_list=[
            [{"role": "user", "content": "Write a poem about asynchronous execution."}],
            [{"role": "user", "content": "Write a poem about asynchronous pirates."}],
        ],
        model="gpt-3.5-turbo-0301",
        temperature=0.3,
        max_tokens=200,
        top_p=1.0,
    )
)

## Experiments

### Direct judge

A hypothesis
- the longer prompt is, the worse result we get

In [ ]:
judge_prompt_0 = "Answer whether the following statement is plausible. Answer with only Yes or No:\n{}"
judge_prompt_1 = "Judge the following statement if it's likely to occur, only answer 'True' or 'False':\n{}"
judge_prompt_1_different_question = "Please judge if the following inference is likely 'True' or 'False'. Only answer 'True' or 'False':\n{}"
judge_prompt_4 = "Judge the following statements if they are likely to occur, only answer 'True' or 'False' for each:\n{}"
judge_prompt_2_cms = "Judge the following statement if it's commonsense, only answer 'True' or 'False':\n{}"
judge_prompt_3_guidance = """Your task is to judge a statement/inference if it's likely to occur. Only answer 'True' or 'False'.
Please not only consider the relevance of two clauses in the statements/inferences, but also the aspect that it refers to, e.g emotion, intention, attribute, etc.
For example,
## Example 1
Statement: PersonX learns to play the trumpet, thus, PersonY wants to applaud PersonX's talent
Answer: True

## Example 2
Statement: PersonX be smart, thus it can be seen about PersonX's attribute that PersonX have always make good grade
Answer: False

## Example 3
Statement: PersonX do not feel well, thus as a result on PersonX's emotion, PersonX only make it through half.
Answer: False

Now, please do your task for each of the following statement(s):
{}"""
judge_prompt_5_guidance = "Judge the below statement/inference if it's likely to occur, only answer 'True' or 'False'. Please not only consider the relevance of two clauses in the statements/inferences, but also the aspect that it refers to, e.g emotion, intention, attribute, etc.\n{}"
judge_prompt_6_guidance = """Your task is to do the following step by step, given the original 'Statement'.
Thinking: Think about the reason if it's likely to occur
Answer: Judge if the statement is likely to occur. Only answer 'True' or 'False'.

For example,
## Example 1: PersonX learns to play the trumpet, thus, PersonY wants to applaud PersonX's talent
Thinking: People tend to applaud other people's effort and talent
Answer: True

## Example 2: PersonX be smart, thus it can be seen about PersonX's attribute that PersonX have a good grade
Thinking: Though "PersonX have a good grade" tends to happen when PersonX is smart, it is a fact rather than an attribute of PersonX
Answer: False

## Example 3: PersonX do not feel well, thus as a result on PersonX's emotion, PersonX only make it through half.
Thinking: "only make it through half" is not related to PersonX's emotion or feeling
Answer: False

Now, please do your task for each of the following statement(s):
{}"""
judge_prompt_7 = """Judge the following statement(s) if it's likely to occur, answer 'True' or 'False' then explain briefly:
Example: PersonX do not feel well, thus as a result on PersonX's emotion, PersonX only make it through half.
Answer: False. "only make it through half" is not related to PersonX's emotion or feeling

Example: {}
Answer: """

conceptualization_prompt = """Your task is to do the following step by step, given the original statement.
1. find a phrase in the statement (phrases in the last clause of the statement are preferable)
2. conceptualize it in a concise manner
3. write down the conceptualized statement formed by direct substitution
4. based on both original and conceptualized statements, judge if the original statement likely to occur by answering 'True' or 'False'. After that, you may briefly explain within one sentence.

For example,

## Example 1
Statement: PersonX learns to play the trumpet, thus, PersonY wants to applaud PersonX's talent
1. the trumpet
2. instrument
3. PersonX learns to play instrument, thus PersonY wants to applaud PersonX's talent
4. True. Because people tend to applaud other people's effort and talent.

## Example 2
Statement: PersonX wins the costume contest, thus, PersonX feel excited
1. the costume contest
2. competition
3. PersonX wins competition, thus PersonX feel excited
4. True. Because it's very happy to win a competition.

## Example 3
Statement: PersonX do not feel well, thus as a result on PersonX's emotion, PersonX only make it through half.
1. only make it through half
2. don't finish
3. PersonX do not feel well, thus as a result on PersonX's emotion, PersonX don't finish
4. False. Because "don't finish" is an action, rather than an emotion.

Now, please do your task for each of the following statement(s):
Statement: {}"""

#### Template 0: ckbp v2 paper

worse than template 1 in both dev and tst. But seem to be less strict (as True/False seems to be more strict than Yes/No), thus try on text-davinci-003

Hmm, it's better, but far from gpt3.5-turbo_template1

Actually, by running these two prompts, we can see how sensitive LLMs are w.r.t different prompt

In [ ]:
bs = 102
predictions = []
start = time.time()

for i in range(len(data)//bs):
    pred = await dispatch_openai_requests(
            messages_list = [[{"role": "user", "content": judge_prompt_0.format(s)}] for s in data[i*bs:(i+1)*bs]],
            max_tokens=3,
        )
    print('\n'.join([x['choices'][0]['message']['content'] for x in pred]))
    predictions.extend([x['choices'][0]['message']['content'] for x in pred])

end = time.time()
print('Running time:', end - start)

#### Template 1: judge_prompt

In [ ]:
bs = 102
predictions = []
start = time.time()

for i in range(len(data)//bs):
    pred = await dispatch_openai_requests(
            messages_list = [[{"role": "user", "content": judge_prompt_1.format(s)}] for s in data[i*bs:(i+1)*bs]],
            max_tokens=3,
        )
    print('\n'.join([x['choices'][0]['message']['content'] for x in pred]))
    predictions.extend([x['choices'][0]['message']['content'] for x in pred])
    time.sleep(10)

end = time.time()
print('Running time:', end - start)

In [ ]:
fail_indices = [i for i, a in enumerate(predictions) if ('True' not in a) and ('False' not in a)]
if len(fail_indices) > 0:
    pred = await dispatch_openai_requests(
            messages_list = [[{"role": "user", "content": judge_prompt_1.format(data[i])}] for i in fail_indices],
            max_tokens=3,
        )
    print('\n'.join([x['choices'][0]['message']['content'] for x in pred]))
    for i, x in zip(fail_indices, pred):
        predictions[i] = x['choices'][0]['message']['content']

In [ ]:
predictions = []

for i in range(len(data)):
    print(data[i])
    pred = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0301",
        messages=[{"role": "user", "content": judge_prompt_1.format(data[i])}],
        max_tokens=4, temperature=0
    )
    print(pred['choices'][0]['message']['content'])
    predictions.append(pred['choices'][0]['message']['content'])
    time.sleep(20)

In [ ]:
bs = 3
for i in range(len(data)//bs+1):
    prompt = judge_prompt_1.format('\n'.join(data[bs*i:bs*(i+1)]))
    gen_chatgpt = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0301",
        messages=[{"role": "user", "content": prompt}],
        # max_tokens=2000,
        temperature=0
    )
    print(gen_chatgpt['choices'][0]['message']['content'])
    break

#### Template 2: judge_prompt_cms

In [ ]:
bs = 102
predictions = []
start = time.time()

for i in range(len(data)//bs):
    pred = await dispatch_openai_requests(
            messages_list = [[{"role": "user", "content": judge_prompt_2_cms.format(s)}] for s in data[i*bs:(i+1)*bs]],
            max_tokens=3,
        )
    print('\n'.join([x['choices'][0]['message']['content'] for x in pred]))
    predictions.extend([x['choices'][0]['message']['content'] for x in pred])
    time.sleep(10)

end = time.time()
print('Running time:', end - start)

#### Template 3 with guidance judge_prompt_guidance

In [ ]:
bs = 20
for i in range(len(data)//bs+1):
    prompt = judge_prompt_3_guidance.format('\n'.join(data[bs*i:bs*(i+1)]))
    gen_chatgpt = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0301",
        messages=[{"role": "user", "content": prompt}],
        # max_tokens=2000,
        temperature=0
    )
    print(gen_chatgpt['choices'][0]['message']['content'])
    time.sleep(20)

Template 4

In [ ]:
bs = 18
for i in range(len(data)//bs+1):
    prompt = judge_prompt_4.format('\n'.join(data[bs*i:bs*(i+1)]))
    gen_chatgpt = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0301",
        messages=[{"role": "user", "content": prompt}],
        # max_tokens=2000,
        temperature=0
    )
    print('\n', gen_chatgpt['choices'][0]['message']['content'])
    time.sleep(20)

In [ ]:
bs = 1
for i in range((len(data)-1)//bs+1):
    prompt = judge_prompt_4.format('\n'.join(data[bs*i:bs*(i+1)]))
    gen_chatgpt = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0301",
        messages=[{"role": "user", "content": prompt}],
        # max_tokens=2000,
        temperature=0
    )
    print(gen_chatgpt['choices'][0]['message']['content'])
    time.sleep(20)

Template 5

In [ ]:
bs = 102
predictions = []
start = time.time()

for i in range(len(data)//bs):
    pred = await dispatch_openai_requests(
            messages_list = [[{"role": "user", "content": judge_prompt_5_guidance.format(s)}] for s in data[i*bs:(i+1)*bs]],
            max_tokens=3,
        )
    print('\n'.join([x['choices'][0]['message']['content'] for x in pred]))
    predictions.extend([x['choices'][0]['message']['content'] for x in pred])
    time.sleep(10)

end = time.time()
print('Running time:', end - start)

Template 6

In [ ]:
bs = 102
predictions = []
start = time.time()

for i in range(len(data)//bs):
    pred = await dispatch_openai_requests(
            messages_list = [[{"role": "user", "content": judge_prompt_6_guidance.format(s)}] for s in data[i*bs:(i+1)*bs]],
            max_tokens=100,
        )
    print('\n\n'.join([x['choices'][0]['message']['content'] for x in pred]))
    predictions.extend([x['choices'][0]['message']['content'] for x in pred])
    time.sleep(10)

end = time.time()
print('Running time:', end - start)

Template 7

In [ ]:
bs = 102
predictions = []
start = time.time()

for i in range(len(data)//bs):
    pred = await dispatch_openai_requests(
            messages_list = [[{"role": "user", "content": judge_prompt_7.format(s)}] for s in data[i*bs:(i+1)*bs]],
            max_tokens=30,
        )
    print('\n\n'.join([x['choices'][0]['message']['content'] for x in pred]))
    predictions.extend([x['choices'][0]['message']['content'] for x in pred])
    time.sleep(10)

end = time.time()
print('Running time:', end - start)

### Zeroshot CoT

In [ ]:
zeroshotCoT_prompt_tq = "Answer whether the statement '{}' is plausible. Let's think step by step, then conclude by answering 'True' or 'False'."
zeroshotCoT_prompt1 = "Judge the statement '{}' if it's likely to occur. Let's think step by step, then conclude by answering 'True' or 'False'." # if add only, it will strictly follow that command, only produce 'True' or 'False' w/o CoT
# the combination zeroshotCoT_prompt_tq + assertion_tq is so baddd!
# naming rule:
# zeroshot_prompt1 = zeroshotCoT_prompt_tq + assertion
# zeroshot_prompt2 = zeroshotCoT_prompt1 + assertion_tq
# zeroshot_prompt3 = zeroshotCoT_prompt1 + assertion

In [ ]:
print(
    openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0301",
        messages=[{"role": "user", "content": zeroshotCoT_prompt_tq.format(data_df['assertion'][0])}],
        max_tokens=100, temperature=0
    )['choices'][0]['message']['content'],
    openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0301",
        messages=[{"role": "user", "content": zeroshotCoT_prompt_tq.format(data_df['assertion'][4])}],
        max_tokens=100, temperature=0
    )['choices'][0]['message']['content'],
    openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0301",
        messages=[{"role": "user", "content": zeroshotCoT_prompt1.format(data_df['assertion_tq'][0])}],
        max_tokens=100, temperature=0
    )['choices'][0]['message']['content'],
    openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0301",
        messages=[{"role": "user", "content": zeroshotCoT_prompt1.format(data_df['assertion_tq'][4])}],
        max_tokens=100, temperature=0
    )['choices'][0]['message']['content'],
    sep='\n'
)

True.
True.
Step 1: Ejaculation is a physical response that occurs when a person reaches sexual climax.
Step 2: Wanting to ejaculate is a psychological desire that may or may not be present during sexual activity.
Step 3: Orgasm is the peak of sexual pleasure and can occur with or without ejaculation.
Step 4: Therefore, it is possible for a person to ejaculate without wanting to or without experiencing an orgasm.

Conclusion: False. Ejaculation does not necessarily indicate a
Step 1: If PersonX enlist
Step 2: As a result, PersonX wants to
Step 3: PersonX wants to fight for country

Conclusion: It is likely to occur that if PersonX enlist, as a result, PersonX wants to fight for country. True.


In [ ]:
print(
    openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0301",
        messages=[{"role": "user", "content": zeroshotCoT_prompt_tq.format(data_df['assertion'][0])}],
        max_tokens=100, temperature=0
    )['choices'][0]['message']['content'],
    openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0301",
        messages=[{"role": "user", "content": zeroshotCoT_prompt_tq.format(data_df['assertion'][4])}],
        max_tokens=100, temperature=0
    )['choices'][0]['message']['content'],
    openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0301",
        messages=[{"role": "user", "content": zeroshotCoT_prompt_tq.format(data_df['assertion_tq'][0])}],
        max_tokens=100, temperature=0
    )['choices'][0]['message']['content'],
    openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0301",
        messages=[{"role": "user", "content": zeroshotCoT_prompt_tq.format(data_df['assertion_tq'][4])}],
        max_tokens=100, temperature=0
    )['choices'][0]['message']['content'],
    sep='\n'
)

True.
True.
As an AI language model, I cannot determine the plausibility of the statement as it is incomplete and lacks context. Please provide more information or rephrase the statement.
As an AI language model, I cannot determine the intention or desire of a person. Therefore, I cannot determine the plausibility of the statement.


In [ ]:
print(
    openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0301",
        messages=[{"role": "user", "content": zeroshotCoT_prompt1.format(data_df['assertion'][0])}],
        max_tokens=100, temperature=0
    )['choices'][0]['message']['content'],
    openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0301",
        messages=[{"role": "user", "content": zeroshotCoT_prompt1.format(data_df['assertion'][4])}],
        max_tokens=100, temperature=0
    )['choices'][0]['message']['content'],
    sep='\n'
)

Step 1: Ejaculation does not always indicate that a person wants an orgasm. It is possible for a person to ejaculate without wanting an orgasm, such as during a medical procedure or due to a reflex action.

Step 2: However, in most cases, ejaculation is a result of sexual stimulation and is often accompanied by the desire for an orgasm.

Step 3: Therefore, it is likely that if PersonX ejaculates, PersonX wants an orgasm.

Conclusion: True.
Step 1: Enlisting in the military is a common way to show a desire to serve one's country.
True.

Step 2: However, there may be other reasons why PersonX enlisted, such as financial benefits or a desire for adventure.
True.

Step 3: Without knowing PersonX's specific motivations for enlisting, it is impossible to definitively say whether or not they want to fight for their country.
True.

Conclusion: The statement "PersonX enlisted, thus, Person


In [ ]:
openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0301",
    messages=[{"role": "user", "content": zeroshotCoT_prompt_tq.format(data_df['assertion'][0])}],
    max_tokens=100, temperature=0
)


<OpenAIObject chat.completion id=chatcmpl-7LRswM6EXKbnVkz2dRUYzNGxOWgxB at 0x7f4f1ea787c0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "As an AI language model, I cannot determine the plausibility of a statement without additional context. However, in general, ejaculation does not necessarily indicate that a person wants an orgasm. Ejaculation can occur without orgasm, and some people may choose to ejaculate without seeking orgasm. It ultimately depends on the individual's preferences and desires.",
        "role": "assistant"
      }
    }
  ],
  "created": 1685346434,
  "id": "chatcmpl-7LRswM6EXKbnVkz2dRUYzNGxOWgxB",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 68,
    "prompt_tokens": 37,
    "total_tokens": 105
  }
}

In [ ]:
openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0301",
    messages=[{"role": "user", "content": "Answer whether the statement '{}' is plausible.".format(data_df['assertion_tq'][0])}],
    max_tokens=100, temperature=0
)


<OpenAIObject chat.completion id=chatcmpl-7LRpiOQR0aL7YC1cVyCwNRaDSFb1R at 0x7f4f1ea57e20> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "As an AI language model, I cannot determine the plausibility of the statement as it is incomplete and lacks context. Please provide more information or rephrase the statement for me to provide a proper answer.",
        "role": "assistant"
      }
    }
  ],
  "created": 1685346234,
  "id": "chatcmpl-7LRpiOQR0aL7YC1cVyCwNRaDSFb1R",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 41,
    "prompt_tokens": 35,
    "total_tokens": 76
  }
}

Try several CoT prompts below, and several ways to place the instruction. Two main problems
- they are not abstraction
- they don't (or me don't know how to) produce a fixed answer term, when run in batch, don't know how to judge automatically!

Also, don't know if using role message is better than normal prompt for gpt-3.5-turbo, not run all dev instance yet!

In [ ]:
role = "You are a commonsense knowledge judge, who gives final answer as 'Yes' or 'No'. Given a statement without any additional context, "+\
    "please focus on the last clause in the statement and judge if it's likely to occur given the context before it."

In [ ]:
cot_prompt_1 = "Judge the following statement if it's likely to occur:\n{}"
role = "You are a commonsense knowledge judge. Given a statement without any additional context, "+\
    "please focus on the last clause in the statement and judge if it's likely to occur given the context before it."
cot_prompt_2 = "Please focus on the last clause in the statement and judge if it's likely to occur given the context before it.\n{}"

In [ ]:
for i in [34]:
    print(data[i])
    pred = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0301",
        messages=[
            # {"role": "system", "content": role},
            {"role": "user", "content": cot_prompt_2.format(data[i])} # + "\n\nLet's think step by step"},
        ],
        # max_tokens=4,
        temperature=0
    )
    print(pred['choices'][0]['message']['content'])
    # predictions.append(pred['choices'][0]['message']['content'])
    # time.sleep(20)

In [ ]:
for i in [34]:
        messages=[
            # {"role": "system", "content": role},
            {"role": "user", "content": cot_prompt_2.format(data[i])}
        ],

# Given ..., likely ...

In [ ]:
for i in [34]:
        messages=[
            # {"role": "system", "content": role},
            {"role": "user", "content": cot_prompt_2.format(data[i])}
        ],

# As an AI language model, I cannot determine the likelihood of an event occurring without additional context. Please provide more information.

In [ ]:
cot_prompt_2 = "Please focus on the last clause in the statement and judge if it's likely to occur given the context before it.\n{}"

for i in [38]:
        messages=[
            # {"role": "system", "content": role},
            {"role": "user", "content": cot_prompt_2.format(data[i])}
        ],

In [ ]:
for i in [38]:
        model="gpt-3.5-turbo-0301",
        messages=[
            {"role": "system", "content": role},
            {"role": "user", "content": data[i] + "\n\nLet's think step by step"},
        ],

In [ ]:
role = "You are a commonsense knowledge judge. Given a statement without any additional context, "+\
    "please focus on the last clause in the statement and judge if it's likely to occur given the context before it. "+\
    "You may think step by step, but at the end, please conclude your answer in short as 'True' or 'False'."

for i in [34]:
        model="gpt-3.5-turbo-0301",
        messages=[
            {"role": "system", "content": role},
            {"role": "user", "content": data[i] + "\n\nLet's think step by step"},
        ],

In [ ]:
role = "You are a commonsense knowledge judge. Given a statement without any additional context, "+\
    "please focus on the last clause in the statement and judge if it's likely to occur given the context before it. "

for i in [34]:
        model="gpt-3.5-turbo-0301",
        messages=[
            {"role": "system", "content": role},
            {"role": "user", "content": data[i] + "\n\nLet's think step by step"},
        ],

In [ ]:
for i in [38]:
        model="gpt-3.5-turbo-0301",
        messages=[
            {"role": "system", "content": role},
            {"role": "user", "content": data[i] + "\n\nLet's think step by step"},
        ],

In [ ]:
messages=[
            {"role": "system", "content": "You are a commonsense knowledge judge"},
            {"role": "user", "content": cot_prompt_1.format(data[i]) + "\nLet's think step by step"},
        ],

In [ ]:
messages=[
            {"role": "user", "content": cot_prompt_1.format(data[i]) + "\nLet's think step by step"}
        ],

# OMG! the answer is totally different, by just changing the position of the term "let's think step by step"

In [ ]:
messages=[
            {"role": "user", "content": cot_prompt_1.format(data[i])},
            {"role": "assistant", "content": "Let's think step by step"}
        ],

### Abstraction

**Why not** abstract the tail into the aspect it expresses, then match with relation?

Will prove that Automated-CoT or normal CoT will not solve CKBP!
While for other commonsense datasets CSQA/StrategyQA, our method is better (of course, human effort is better)

Very unpredictable! more exemplar, worse judgment! (see text-davinci-003 draft experiment)

In [ ]:
old_conceptualization_prompt = """Your task is to do the following step by step, given the original statement.
1. find a phrase in the statement (phrases in the last clause of the statement are preferable)
2. conceptualize it in a concise manner
3. write down the conceptualized statement formed by direct substitution
4. based on both original and conceptualized statements, judge if the original statement likely to occur by answering 'True' or 'False'. After that, you may briefly explain within one sentence.

For example,

## Example 1
Statement: PersonX learns to play the trumpet, thus, PersonY wants to applaud PersonX's talent
1. the trumpet
2. instrument
3. PersonX learns to play instrument, thus PersonY wants to applaud PersonX's talent
4. True. Because people tend to applaud other people's effort and talent.

## Example 2
Statement: PersonX wins the costume contest, thus, PersonX feel excited
1. the costume contest
2. competition
3. PersonX wins competition, thus PersonX feel excited
4. True. Because it's very happy to win a competition.

## Example 3
Statement: PersonX do not feel well, thus as a result on PersonX's emotion, PersonX only make it through half.
1. only make it through half
2. don't finish
3. PersonX do not feel well, thus as a result on PersonX's emotion, PersonX don't finish
4. False. Because "don't finish" is an action, rather than an emotion.

Now, please do your task for each of the following statement(s):
{}"""

conceptualization_prompt_5exemplars = """Please read the convention, task description, examples, then do the task:
Convention: There are three dimensions ['persona', 'event', 'mental state']. All ordered dimension pairs are valid, except ('event', 'persona') and ('event', 'mental state') are invalid.

Task description: Given an inference and its corresponding dimension D_i and its two main clauses H and T, you are required to do the following step by step:
1. Focus on the last clause T, determine which dimension best describe what the clause expresses. Denote the dimension as D_c.
2. Judge if the pair (D_c, D_i) is valid. Answer 'Yes' or 'No'.
3. Judge if the main clauses' meaning is all clear/non-ambiguous. Answer 'Yes' or 'No'.
4. Based on result in 2, 3, and the semantics of the inference, judge if the inference is likely 'True' or 'False'. Note that if result of either 2 or 3 is 'No', then result of 4 should be 'False'.

Examples:
* Example 1
Inference: PersonX learns to play the trumpet, thus, PersonY wants to applaud PersonX's talent
D_i = 'event', H = 'PersonX learns to play the trumpet', T = 'PersonY wants to applaud PersonX's talent'
1. D_c = 'event'
2. Yes (because the ordered pair (D_c, D_i) = ('event', 'event') is valid)
3. Yes
4. True. Because 1) "PersonX learns to play the trumpet" means PersonX are talent and effortful in learning musical instrument, and 2) people tend to applaud other people's effort and talent.

* Example 2
Inference: PersonX eat most of those, thus it can be seen about PersonX's intention that, PersonX love curly fries
D_i = 'event', H = 'PersonX eat most of those', T = 'PersonX love curly fries'
1. D_c = 'mental state'
2. Yes (because the ordered pair (D_c, D_i) = ('mental state', 'event') is valid)
3. Yes
4. False. Because 'love curly fries' expresses PersonX's preference rather than intention.

* Example 3
Inference: PersonX do not feel well, thus as a result on PersonX's emotion, PersonX only make it through half.
D_i = 'mental state', H = 'PersonX do not feel well', T = 'PersonX only make it through half'
1. D_c = 'event'
2. No (because the ordered pair (D_c, D_i) = ('event', 'mental state') is not valid)
3. Yes
4. False. Because, result of 2 is No.

* Example 4
Inference: Before PersonX really empathize with PersonY, PersonX feel.
D_i = 'event', H = 'PersonX really empathize with PersonY', T = 'PersonX feel'
1. D_c = 'mental state'
2. Yes (because the ordered pair (D_c, D_i) = ('mental state', 'event') is valid)
3. No (T is unclear about what 'PersonX feel')
4. False. Because result of 3 is No.

* Example 5
Inference: PersonX have a traumatic incident, thus it can be seen about PersonX's attribute that PersonX be introvert
D_i = 'persona', H = 'PersonX have a traumatic incident', T = 'PersonX be introvert'
1. D_c = 'persona'
2. Yes (because the ordered pair (D_c, D_i) = ('persona', 'persona') is valid)
3. Yes
4. False. Because having a traumatic incident likely result from unfortune or carelessness, while being introvert is not relevant.

Now, please do your task for (each of) the following statement(s):
Inference: {}
D_i = '{}', H = '{}', T = '{}'"""


conceptualization_prompt = """Please read the convention, task description, examples, then do the task:
Convention: There are three dimensions ['persona', 'event', 'mental state']. All ordered dimension pairs are valid, except ('event', 'persona') and ('event', 'mental state') are invalid.

Task description: Given an inference and its corresponding dimension D_i and its two main clauses H and T, you are required to do the following step by step:
1. Focus on the last clause T, determine which dimension best describe what the clause expresses. Denote the dimension as D_c.
2. Judge if the ordered pair (D_c, D_i) is valid. Answer 'Yes' or 'No'.
3. Judge if the main clauses' meaning is all clear/non-ambiguous. Answer 'Yes' or 'No'.
4. Based on result in 2, 3, and the semantics of the inference, judge if the inference is likely 'True' or 'False'. Note that if result of either 2 or 3 is 'No', then result of 4 should be 'False'.

Examples:
* Example 1
Inference: PersonX learns to play the trumpet, thus, PersonY wants to applaud PersonX's talent
D_i = 'event', H = 'PersonX learns to play the trumpet', T = 'PersonY wants to applaud PersonX's talent'
1. D_c = 'event'
2. Yes (because the ordered pair (D_c, D_i) = ('event', 'event') is valid)
3. Yes
4. True. Because 1) "PersonX learns to play the trumpet" means PersonX are talent and effortful in learning musical instrument, and 2) people tend to applaud other people's effort and talent.

* Example 2
Inference: PersonX do not feel well, thus as a result on PersonX's emotion, PersonX only make it through half.
D_i = 'mental state', H = 'PersonX do not feel well', T = 'PersonX only make it through half'
1. D_c = 'event'
2. No (because the ordered pair (D_c, D_i) = ('event', 'mental state') is not valid)
3. Yes
4. False. Because, result of 2 is No.

Now, please do your task for (each of) the following statement(s):
Inference: {}
D_i = '{}', H = '{}', T = '{}'"""

conceptualization_prompt_batch_input = """Please read the convention, task description, examples, then do the task:
Convention: There are three dimensions ['persona', 'event', 'mental state']. All ordered dimension pairs are valid, except ('event', 'persona') and ('event', 'mental state') are invalid.

Task description: Given an inference and its corresponding dimension D_i and its two main clauses H and T, you are required to do the following step by step:
1. Focus on the last clause T, determine which dimension best describe what the clause expresses. Denote the dimension as D_c.
2. Judge if the ordered pair (D_c, D_i) is valid. Answer 'Yes' or 'No'.
3. Judge if the main clauses' meaning is all clear/non-ambiguous. Answer 'Yes' or 'No'.
4. Based on result in 2, 3, and the semantics of the inference, judge if the inference is likely 'True' or 'False'. Note that if result of either 2 or 3 is 'No', then result of 4 should be 'False'.

Examples:
* Example 1
Inference: PersonX learns to play the trumpet, thus, PersonY wants to applaud PersonX's talent,
D_i = 'event', H = 'PersonX learns to play the trumpet', T = 'PersonY wants to applaud PersonX's talent'
1. D_c = 'event'
2. Yes (because the ordered pair (D_c, D_i) = ('event', 'event') is valid)
3. Yes
4. True. Because 1) "PersonX learns to play the trumpet" means PersonX are talent and effortful in learning musical instrument, and 2) people tend to applaud other people's effort and talent.

* Example 2
Inference: PersonX do not feel well, thus as a result on PersonX's emotion, PersonX only make it through half.,
D_i = 'mental state', H = 'PersonX do not feel well', T = 'PersonX only make it through half'
1. D_c = 'event'
2. No (because the ordered pair (D_c, D_i) = ('event', 'mental state') is not valid)
3. Yes
4. False. Because, result of 2 is No.

Now, please do your task for (each of) the following statement(s):
"""
conceptualization_prompt_batch_input_tail = "Inference: {}, D_i = '{}', H = '{}', T = '{}'"

gpt-3.5-turbo is so stupid in following rule! Need to take the rule explicit

In [ ]:
conceptualization_prompt_chatgpt = """Please read the convention, task description, examples, then do the task:
Convention: There are three dimensions ['persona', 'event', 'mental state'].

Task description: Given an inference and its corresponding dimension D_i and its two main clauses H and T, you are required to do the following step by step:
1. Focus on the last clause T, determine which dimension best describe what the clause expresses. Denote the dimension as D_c.
2. If the ordered pair (D_c, D_i) is ('event', 'persona') or ('event', 'mental state') then write 'No', otherwise write 'Yes'.
3. Judge if the main clauses' meaning is all clear/non-ambiguous. Answer 'Yes' or 'No'.
4. Based on result in 2, 3, and the semantics of the inference, judge if the inference is likely 'True' or 'False'. Note that if result of either 2 or 3 is 'No', then result of 4 should be 'False'.

Examples:
* Example 1
Inference: PersonX learns to play the trumpet, thus, PersonY wants to applaud PersonX's talent
D_i = 'event', H = 'PersonX learns to play the trumpet', T = 'PersonY wants to applaud PersonX's talent'
1. D_c = 'event'
2. Yes
3. Yes
4. True. Because 1) "PersonX learns to play the trumpet" means PersonX are talent and effortful in learning musical instrument, and 2) people tend to applaud other people's effort and talent.

* Example 2
Inference: PersonX eat most of those, thus it can be seen about PersonX's intention that, PersonX love curly fries
D_i = 'event', H = 'PersonX eat most of those', T = 'PersonX love curly fries'
1. D_c = 'mental state'
2. Yes
3. Yes
4. False. Because 'love curly fries' expresses PersonX's preference rather than intention.

* Example 3
Inference: PersonX do not feel well, thus as a result on PersonX's emotion, PersonX only make it through half.
D_i = 'mental state', H = 'PersonX do not feel well', T = 'PersonX only make it through half'
1. D_c = 'event'
2. No
3. Yes
4. False. Because, result of 2 is No.

* Example 4
Inference: Before PersonX really empathize with PersonY, PersonX feel.
D_i = 'event', H = 'PersonX really empathize with PersonY', T = 'PersonX feel'
1. D_c = 'mental state'
2. Yes
3. No (T is unclear about what 'PersonX feel')
4. False. Because result of 3 is No.

* Example 5
Inference: PersonX have a traumatic incident, thus it can be seen about PersonX's attribute that PersonX be introvert
D_i = 'persona', H = 'PersonX have a traumatic incident', T = 'PersonX be introvert'
1. D_c = 'persona'
2. Yes
3. Yes
4. False. Because having a traumatic incident likely result from unfortune or carelessness, while being introvert is not relevant.

Now, please do your task for (each of) the following statement(s):
Inference: {}
D_i = '{}', H = '{}', T = '{}'"""

conceptualization_prompt_chatgpt_2 = """Please read the task description and examples, then do the task:
Task description: Given an inference, its aspect D, and its two main clauses H and T, you are required to do the following step by step:
1. Determine if clause T expresses about a/an D of the subject. Answer 'Yes' or 'No'.
2. Judge if the meanings of H and T are all clear/non-ambiguous. Answer 'Yes' or 'No'.
3. If result of either 1 or 2 is 'No', please answer 'False'. Otherwise, judge if the inference is likely 'True' or 'False'.

Examples:
* Example 1
Inference: PersonX learns to play the trumpet, thus, PersonY wants to applaud PersonX's talent
D = 'event', H = 'PersonX learns to play the trumpet', T = 'PersonY wants to applaud PersonX's talent'
1. Yes. Because 'PersonY wants to applaud PersonX's talent' expresses an 'event' of PersonY
2. Yes.
3. True. Because 1) "PersonX learns to play the trumpet" means PersonX are talent and effortful in learning musical instrument, and 2) people tend to applaud other people's effort and talent.

* Example 2
Inference: PersonX eat most of those, thus as a result on PersonX's emotion, PersonX feel hungry
D = 'mental state', H = 'PersonX eat most of those', T = 'PersonX feel hungry'
1. Yes. Because 'PersonX feel hungry' expresses a 'mental state' of PersonX
2. Yes.
3. False. Because it's unlikely that PersonX still feel hungry after eat a lot.

* Example 3
Inference: PersonX do not feel well, thus as a result on PersonX's emotion, PersonX only make it through half.
D = 'mental state', H = 'PersonX do not feel well', T = 'PersonX only make it through half'
1. No. Because 'PersonX only make it through half' doesn't expresses an 'mental state' of PersonX
2. Yes.
3. False. Because, result of 1 is No.

* Example 4
Inference: Before PersonX really empathize with PersonY, PersonX feel.
D = 'event', H = 'PersonX really empathize with PersonY', T = 'PersonX feel'
1. Yes. Because 'PersonX feel' expresses an 'event' of PersonX
2. No. Because the meaning of 'PersonX feel' is unclear/ambiguous
3. False. Because result of 2 is No.

* Example 5
Inference: PersonX have a traumatic incident, thus it can be seen about PersonX's attribute that PersonX be introvert
D = 'persona', H = 'PersonX have a traumatic incident', T = 'PersonX be introvert'
1. Yes. Because 'PersonX be introvert' expresses a persona of PersonX
2. Yes.
3. False. Because having a traumatic incident likely result from unfortune or carelessness, while being introvert is not relevant.

Now, please do your task for (each of) the following statement(s):
Inference: {}
D = '{}', H = '{}', T = '{}'"""

conceptualization_prompt_chatgpt_symbolic = """Please read the convention, task description, examples, then do the task:
Convention: There are three dimensions S = ['persona', 'event', 'mental state'] and a binary operator R: S x S as follow
R('persona', 'persona') = R('persona', 'event') = R('persona', 'mental state') = R('event', 'event') = R('mental state', 'persona') = R('mental state', 'event') = R('mental state', 'mental state') = 'Yes'
R('event', 'persona') = R('event', 'mental state') = 'No'

Task description: Given an inference and its corresponding dimension D_i and its two main clauses H and T, you are required to do the following step by step:
1. Focus on the last clause T, determine which dimension best describe what the clause expresses. Denote the dimension as D_c.
2. Compute R(D_c, D_i)
3. Judge if the main clauses' meaning is all clear/non-ambiguous. Answer 'Yes' or 'No'.
4. Based on result in 2, 3, and the semantics of the inference, judge if the inference is likely 'True' or 'False'. Note that if result of either 2 or 3 is 'No', then result of 4 should be 'False'.

Examples:
* Example 1
Inference: PersonX learns to play the trumpet, thus, PersonY wants to applaud PersonX's talent
D_i = 'event', H = 'PersonX learns to play the trumpet', T = 'PersonY wants to applaud PersonX's talent'
1. D_c = 'event'
2. R(D_c, D_i) = R('event', 'event') = Yes
3. Yes
4. True. Because 1) "PersonX learns to play the trumpet" means PersonX are talent and effortful in learning musical instrument, and 2) people tend to applaud other people's effort and talent.

* Example 2
Inference: PersonX eat most of those, thus it can be seen about PersonX's intention that, PersonX love curly fries
D_i = 'event', H = 'PersonX eat most of those', T = 'PersonX love curly fries'
1. D_c = 'mental state'
2. R(D_c, D_i) = R('mental state', 'event') = Yes
3. Yes
4. False. Because 'love curly fries' expresses PersonX's preference rather than intention.

* Example 3
Inference: PersonX do not feel well, thus as a result on PersonX's emotion, PersonX only make it through half.
D_i = 'mental state', H = 'PersonX do not feel well', T = 'PersonX only make it through half'
1. D_c = 'event'
2. R(D_c, D_i) = R('event', 'mental state') = No
3. Yes
4. False. Because, result of 2 is No.

* Example 4
Inference: Before PersonX really empathize with PersonY, PersonX feel.
D_i = 'event', H = 'PersonX really empathize with PersonY', T = 'PersonX feel'
1. D_c = 'mental state'
2. R(D_c, D_i) = R('mental state', 'event') = Yes
3. No (T is unclear about what 'PersonX feel')
4. False. Because result of 3 is No.

* Example 5
Inference: PersonX have a traumatic incident, thus it can be seen about PersonX's attribute that PersonX be introvert
D_i = 'persona', H = 'PersonX have a traumatic incident', T = 'PersonX be introvert'
1. D_c = 'persona'
2. R(D_c, D_i) = R('persona', 'persona') = Yes
3. Yes
4. False. Because having a traumatic incident likely result from unfortune or carelessness, while being introvert is not relevant.


Now, please do your task for (each of) the following statement(s):
Inference: {}
D_i = '{}', H = '{}', T = '{}'"""


conceptualization_prompt_chatgpt_2_different_question = """Please read the task description and examples, then do the task:
Task description: Given an inference, its aspect D, and its two main clauses H and T, you are required to do the following step by step:"""
1. Determine if clause T expresses about a/an D of the subject. Answer 'Yes' or 'No'. # unclear question! Need to do analysis
"""
2. Judge if the meanings of H and T are all clear/non-ambiguous. Answer 'Yes' or 'No'.
3. If result of either 1 or 2 is 'No', please answer 'False'. Otherwise, judge if the inference is likely 'True' or 'False'.

Examples:
* Example 1
Inference: PersonX learns to play the trumpet, thus, PersonY wants to applaud PersonX's talent
D = 'event', H = 'PersonX learns to play the trumpet', T = 'PersonY wants to applaud PersonX's talent'
1. Yes. Because 'PersonY wants to applaud PersonX's talent' expresses an 'event' of PersonY
2. Yes.
3. True. Because 1) "PersonX learns to play the trumpet" means PersonX are talent and effortful in learning musical instrument, and 2) people tend to applaud other people's effort and talent.

* Example 2
Inference: PersonX eat most of those, thus as a result on PersonX's emotion, PersonX feel hungry
D = 'mental state', H = 'PersonX eat most of those', T = 'PersonX feel hungry'
1. Yes. Because 'PersonX feel hungry' expresses a 'mental state' of PersonX
2. Yes.
3. False. Because it's unlikely that PersonX still feel hungry after eat a lot.

* Example 3
Inference: PersonX do not feel well, thus as a result on PersonX's emotion, PersonX only make it through half.
D = 'mental state', H = 'PersonX do not feel well', T = 'PersonX only make it through half'
1. No. Because 'PersonX only make it through half' doesn't expresses an 'mental state' of PersonX
2. Yes.
3. False. Because, result of 1 is No.

* Example 4
Inference: Before PersonX really empathize with PersonY, PersonX feel.
D = 'event', H = 'PersonX really empathize with PersonY', T = 'PersonX feel'
1. Yes. Because 'PersonX feel' expresses an 'event' of PersonX
2. No. Because the meaning of 'PersonX feel' is unclear/ambiguous
3. False. Because result of 2 is No.

* Example 5
Inference: PersonX have a traumatic incident, thus it can be seen about PersonX's attribute that PersonX be introvert
D = 'persona', H = 'PersonX have a traumatic incident', T = 'PersonX be introvert'
1. Yes. Because 'PersonX be introvert' expresses a persona of PersonX
2. Yes.
3. False. Because having a traumatic incident likely result from unfortune or carelessness, while being introvert is not relevant.

Now, please do your task for (each of) the following statement(s):
Inference: {}
D = '{}', H = '{}', T = '{}'"""


concept template 2

In [ ]:
bs = 102 # len(data_df)
predictions = []

# bs = 1; for b in [38]:
for b in range(len(data_df)//bs):
    pred = await dispatch_openai_requests(
            messages_list = [
                [{
                "role": "user",
                "content": conceptualization_prompt_chatgpt_2.format(
                    data_df['assertion'][i],
                    data_df['dim'][i],
                    data_df['head'][i],
                    data_df['tail'][i])
                }]
                for i in range(b*bs, min((b+1)*bs, len(data_df)))
            ],
            max_tokens=100,
        )
    print('\n'.join([x['choices'][0]['message']['content'] for x in pred]))
    predictions.extend([x['choices'] for x in pred])
    time.sleep(10)


In [ ]:
print(predictions)

concept template 1

In [ ]:
bs = 102 # len(data_df)
predictions = []

# bs = 1; for b in [38]:
for b in range(len(data_df)//bs):
    pred = await dispatch_openai_requests(
            messages_list = [
                [{
                "role": "user",
                "content": conceptualization_prompt_chatgpt.format(
                    data_df['assertion'][i],
                    data_df['dim'][i],
                    data_df['head'][i],
                    data_df['tail'][i])
                }]
                for i in range(b*bs, min((b+1)*bs, len(data_df)))
            ],
            max_tokens=100,
        )
    print('\n'.join([x['choices'][0]['message']['content'] for x in pred]))
    predictions.extend([x['choices'] for x in pred])
    time.sleep(10)

In [ ]:
print(predictions)

In [ ]:
for i, s in enumerate(predictions):
    answer = ''
    s = s[0]['message']['content']
    for l in s.split('\n'):
        if l.startswith('4.'):
            answer = l[3:]
            break
    print(answer)

symbolic

In [ ]:
predictions = []
bs = 1

for i in range((len(data)-1)//bs + 1):
    p = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0301",
        messages=[{"role": "user", "content": conceptualization_prompt_chatgpt_symbolic.format(
            data_df['assertion'][i],
            data_df['dim'][i],
            data_df['head'][i],
            data_df['tail'][i])
        }],
        max_tokens=100,
        temperature=0
    )
    print(data_df['assertion'][i])
    print(p['choices'][0]['message']['content'])
    predictions.extend(p['choices'])
    time.sleep(20)

In [ ]:
for i, s in enumerate(predictions):
    answer = ''
    s = s['message']['content']
    for l in s.split('\n'):
        if l.startswith('4.'):
            answer = l[3:]
            break
    print(answer)

testing

In [ ]:
predictions = []
bs = 1

for i in [38]:
# for i in range((len(data)-1)//bs + 1):
    p = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0301",
        messages=[{"role": "user", "content": conceptualization_prompt_chatgpt.format(
            data_df['assertion'][i],
            data_df['dim'][i],
            data_df['head'][i],
            data_df['tail'][i])
        }],
        max_tokens=100,
        temperature=0
    )
    print(p['choices'])
    predictions.extend(p['choices'])
    break
    time.sleep(20)

In [ ]:
pred = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0301",
    messages=[
        # {"role": "system", "content": role},
        {"role": "user", "content": conceptualization_prompt.format(data[34], 'mental state', 'PersonX prevent PersonY', 'PersonX never reach out to anyone')} # + "\n\nLet's think step by step"},
    ],
    # max_tokens=4,
    temperature=0
)
print(pred['choices'][0]['message']['content'])
pred

In [ ]:
{"role": "user", "content": conceptualization_prompt.format(data[34], 'emotion')} # + "\n\nLet's think step by step"},

# 1. action
# 2. Yes (because the aspect-dimension pair ('action', 'emotion') is valid)
# 3. No
# 4. Likely True. Because the inference suggests that PersonX's emotion is negatively affected by preventing PersonY, which may lead to a lack of desire to reach out to anyone.

In [ ]:
bs = 102
predictions = []
start = time.time()

for i in range(len(data)//bs):
    pred = await dispatch_openai_requests(
            messages_list = [[{"role": "user", "content": conceptualization_prompt.format(s)}] for s in data[i*bs:(i+1)*bs]],
            max_tokens=100,
        )
    print('\n\n'.join([x['choices'][0]['message']['content'] for x in pred]))
    predictions.extend([x['choices'][0]['message']['content'] for x in pred])
    time.sleep(10)

end = time.time()
print('Running time:', end - start)

In [ ]:
for i, s in enumerate(predictions):
    answer = ''
    for l in s.split('\n'):
        if l.startswith('4.'):
            answer = l
            break
    print(i, answer)

In [ ]:
print(gen_chatgpt['choices'][0]['message']['content'])

### MoE

Mixture of Expert or Mixture of Constraint

In [ ]:
moe_q1_prompt1 = "Which aspect (among three options 1. event/activity, 2. persona, 3. mental state) of the subject does the clause '{}' express. Answer the choice only."
moe_q1_prompt2 = "Determine if clause '{}' expresses an event or activity of the subject. Answer 'Yes' or 'No' only."

moe_q2_prompt1 = "Judge if the meaning of the clauses '{}' and '{}' are all clear. Answer 'Yes' or 'No' only."
moe_q2_prompt2 = """Which one of the following two statements make more sense:
0. Two clauses '{0}' and '{1}' all have clear meaning.
1. One of two following clauses '{0}' and '{1}' has ambiguous meaning.
Answer 0 or 1 only."""

moe_q3_prompt1 = "Judge if the event '{}' likely occurs after the event '{}'. Answer 'Yes' or 'No' only."    # pretty unstable
moe_q3_prompt1_reverse = "Judge if the event '{}' likely occurs before the event '{}'. Answer 'Yes' or 'No' only."   # pretty unstable
moe_q3_prompt2 = "Given 2 events in a story: A. '{}', B. '{}'. Is A likely to occur after B? Answer 'Yes' or 'No' only."
moe_q3_prompt3 = """Which one of the following two statements is more plausible:
0. {0} before {1}
1. {0} after {1}
Answer 0 or 1 only.""" # tail [] head, 1 -> after, 0 -> before

moe_q4_prompt1 = "Judge if the event '{}' causes the event '{}'. Answer 'Yes' or 'No' only." # need to observe on dev set. Not easy to judge, thus not System 1 thinking

In [ ]:
i = 0
s = data_df['tail'][i]
pred = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0301",
    messages=[{"role": "user", "content": moe_q1_prompt1.format(s)}],
    max_tokens=4, temperature=0
)

print(s)
pred

In [ ]:
bs = 10
for i in range(0, len(data)//bs+1):
    print(i)
    pred = await dispatch_openai_requests(
            messages_list = [[{"role": "user", "content": moe_q1_prompt1.format(s)}] for s in data_df['tail'][i*bs:(i+1)*bs]],
            max_tokens=4,
        )
    print('\n'.join([x['choices'][0]['message']['content'] for x in pred]))
    predictions.extend([x['choices'][0]['message']['content'] for x in pred])
    time.sleep(5)

In [ ]:
bs = 5
for i in range(0, len(data)//bs+1):
    while True:
        try:
            print(i)
            pred = await dispatch_openai_requests(
                    messages_list = [[{"role": "user", "content": moe_q2_prompt1.format(s)}] for s in data_df['tail'][i*bs:(i+1)*bs]],
                    max_tokens=1,
                )
            print('\n'.join([x['choices'][0]['message']['content'] for x in pred]))
            predictions.extend([x['choices'][0]['message']['content'] for x in pred])
            time.sleep(1)
            break
        except Exception as e:
            print(e)
            time.sleep(10)

In [ ]:
bs = 5
for i in range(0, len(data)//bs+1):
    while True:
        try:
            print(i)
            pred = await dispatch_openai_requests(
                    messages_list = [[{"role": "user", "content": moe_q2_prompt1.format(s)}] for s in data_df['head'][i*bs:(i+1)*bs]],
                    max_tokens=1,
                )
            print('\n'.join([x['choices'][0]['message']['content'] for x in pred]))
            predictions.extend([x['choices'][0]['message']['content'] for x in pred])
            time.sleep(1)
            break
        except Exception as e:
            print(e)
            time.sleep(10)

In [ ]:
i = 60 # 61 or 68
pred = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0301",
    messages=[{"role": "user", "content": moe_q3_prompt1.format(data_df['tail'][i], data_df['head'][i])}],
    max_tokens=50, temperature=0
)

print(moe_q3_prompt3.format(data_df['tail'][i], data_df['head'][i]))
pred

Which one of the following two statements is more plausible:
0. PersonX belief be true before PersonX believe thing
1. PersonX belief be true after PersonX believe thing
Answer 0 or 1 only.


<OpenAIObject chat.completion id=chatcmpl-7JP1BtxcUdJLHqVRnp8zrmYaqag4X at 0x7fbd28254040> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "As an AI language model, I cannot determine the likelihood of an event occurring. However, if PersonX believes something, it is possible that their belief is true or false. Therefore, the answer is 'uncertain'.",
        "role": "assistant"
      }
    }
  ],
  "created": 1684858757,
  "id": "chatcmpl-7JP1BtxcUdJLHqVRnp8zrmYaqag4X",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 44,
    "prompt_tokens": 38,
    "total_tokens": 82
  }
}

In [ ]:
pred = await dispatch_openai_requests(
        messages_list = [[{"role": "user", "content": moe_q3_prompt3.format(data_df['tail'][i], data_df['head'][i])}] \
                         for i in range(0, 102)
                         ],
        max_tokens=4,
    )
print('\n'.join([x['choices'][0]['message']['content'] for x in pred]))

In [ ]:
bs = 5
for i in range(0, len(data)//bs+1):
    while True:
        try:
            print(i, '/')
            pred = await dispatch_openai_requests(
                    messages_list = [[{"role": "user", "content": moe_q3_prompt3.format(data_df['tail'][b], data_df['head'][b])}] \
                                     for b in range(i*bs, min((i+1)*bs, len(data_df)))
                                    ],
                    max_tokens=4,
                )
            print('\n'.join([x['choices'][0]['message']['content'] for x in pred]))
            predictions.extend([x['choices'][0]['message']['content'] for x in pred])
            time.sleep(1)
            break
        except Exception as e:
            print(e)
            time.sleep(10)

### KATE

In [ ]:
kate_prompt_0 = "Answer whether the following statements plausible. Answer with only Yes or No:\n{}"
kate_prompt_1 = "Judge the following statements if they are likely to occur, only answer 'True' or 'False':\n{}"

In [ ]:
s = 22
exemplars = '\n\n'.join([
    'Statement: {}\nAnswer: {}'.format(tst_kate_exemplars['kate_pos_0_tq'][s], 'Yes'),
    'Statement: {}\nAnswer: {}'.format(tst_kate_exemplars['kate_neg_0_tq'][s], 'No'),
    'Statement: {}\nAnswer: {}'.format(tst_data_df['assertion_tq'][s], ''),
])
print(kate_prompt_0.format(exemplars))
pred = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0301",
    messages=[{"role": "user", "content": kate_prompt_0.format(exemplars)}],
    max_tokens=2, temperature=0
)
pred['choices'][0]['message']['content']

Answer whether the following statements plausible. Answer with only Yes or No:
Statement: If PersonX come early, as a result, PersonX will, PersonX go home
Answer: Yes

Statement: If PersonX go home early, as a result, PersonX will, PersonX come day
Answer: No

Statement: If PersonX start early, as a result, PersonX will, PersonX go home early
Answer: 


'Yes'

In [ ]:
s = 22
exemplars = '\n\n'.join([
    'Statement: {}\nAnswer: {}'.format(tst_kate_exemplars['kate_pos_0'][s], 'Yes'),
    'Statement: {}\nAnswer: {}'.format(tst_kate_exemplars['kate_neg_0'][s], 'No'),
    'Statement: {}\nAnswer: {}'.format(tst_data_df['assertion'][s], ''),
])
print(kate_prompt_0.format(exemplars))
pred = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0301",
    messages=[{"role": "user", "content": kate_prompt_0.format(exemplars)}],
    max_tokens=2, temperature=0
)
pred['choices'][0]['message']['content']

Answer whether the following statements plausible. Answer with only Yes or No:
Statement: PersonX come early, thus as an result, PersonX go home
Answer: Yes

Statement: PersonX go home early, thus as an result, PersonX come day
Answer: No

Statement: PersonX start early, thus as an result, PersonX go home early
Answer: 


'Yes'

In [ ]:
s = 22
exemplars = '\n\n'.join([
    'Statement: {}\nAnswer: {}'.format(tst_kate_exemplars['kate_pos_0'][s], 'Yes'),
    'Statement: {}\nAnswer: {}'.format(tst_kate_exemplars['kate_neg_0'][s], 'No'),
    'Statement: {}\nAnswer: {}'.format(tst_data_df['assertion'][s], ''),
])
print(kate_prompt_1.format(exemplars))
pred = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0301",
    messages=[{"role": "user", "content": kate_prompt_0.format(exemplars)}],
    max_tokens=2, temperature=0
)
pred['choices'][0]['message']['content']

Judge the following statements if they are likely to occur, only answer 'True' or 'False':
Statement: PersonX come early, thus as an result, PersonX go home
Answer: Yes

Statement: PersonX go home early, thus as an result, PersonX come day
Answer: No

Statement: PersonX start early, thus as an result, PersonX go home early
Answer: 


'Yes'

In [ ]:
for s in [0, 22]:
    exemplars = '\n\n'.join([
        'Statement: {}\nAnswer: {}'.format(tst_kate_exemplars['kate_pos_0_tq'][s], 'Yes'),
        'Statement: {}\nAnswer: {}'.format(tst_kate_exemplars['kate_neg_0_tq'][s], 'No'),
        'Statement: {}\nAnswer: {}'.format(tst_data_df['assertion_tq'][s], ''),
    ])
    print(kate_prompt_0.format(exemplars))

    pred = openai.Completion.create(
        model="text-davinci-003",
        prompt=[kate_prompt_0.format(exemplars)],
        max_tokens=4, temperature=0
    )
    print(pred)

Answer whether the following statements plausible. Answer with only Yes or No:
Statement: PersonX be real, thus as an result, PersonY argue PersonZ about it
Answer: Yes

Statement: PersonX come to talk to PersonZ, thus as an result, PersonY become enraged
Answer: No

Statement: If PersonX argue with PersonY, as a result, PersonY or others will, PersonY be not a PersonZ
Answer: 
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " No"
    }
  ],
  "created": 1685531442,
  "id": "cmpl-7ME0wp301i7vJFzKrirJKeTere0jl",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 1,
    "prompt_tokens": 97,
    "total_tokens": 98
  }
}
Answer whether the following statements plausible. Answer with only Yes or No:
Statement: PersonX come early, thus as an result, PersonX go home
Answer: Yes

Statement: PersonX go home early, thus as an result, PersonX come day
Answer: No

Statement: If PersonX st

In [ ]:
for s in [0, 22]:
    exemplars = '\n\n'.join([
        'Statement: {}\nAnswer: {}'.format(tst_kate_exemplars['kate_pos_0'][s], 'Yes'),
        'Statement: {}\nAnswer: {}'.format(tst_kate_exemplars['kate_neg_0'][s], 'No'),
        'Statement: {}\nAnswer: {}'.format(tst_data_df['assertion'][s], ''),
    ])
    print(kate_prompt_0.format(exemplars))

    pred = openai.Completion.create(
        model="text-davinci-003",
        prompt=[kate_prompt_0.format(exemplars)],
        max_tokens=4, temperature=0
    )
    print(pred)

Answer whether the following statements plausible. Answer with only Yes or No:
Statement: PersonX be real, thus as an result, PersonY argue PersonZ about it
Answer: Yes

Statement: PersonX come to talk to PersonZ, thus as an result, PersonY become enraged
Answer: No

Statement: PersonX argue with PersonY, thus as an result, PersonY be not a PersonZ
Answer: 
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " No"
    }
  ],
  "created": 1685531441,
  "id": "cmpl-7ME0v6gOYsmVodzMVPUmt2PjBrtVN",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 1,
    "prompt_tokens": 91,
    "total_tokens": 92
  }
}
Answer whether the following statements plausible. Answer with only Yes or No:
Statement: PersonX come early, thus as an result, PersonX go home
Answer: Yes

Statement: PersonX go home early, thus as an result, PersonX come day
Answer: No

Statement: PersonX start early, thus as an re

In [ ]:
for s in [0, 22]:
    exemplars = '\n\n'.join([
        'Statement: {}\nAnswer: {}'.format(tst_kate_exemplars['kate_pos_0'][s], 'True'),
        'Statement: {}\nAnswer: {}'.format(tst_kate_exemplars['kate_neg_0'][s], 'False'),
        'Statement: {}\nAnswer: {}'.format(tst_data_df['assertion'][s], ''),
    ])
    print(kate_prompt_1.format(exemplars))

    pred = openai.Completion.create(
        model="text-davinci-003",
        prompt=[kate_prompt_0.format(exemplars)],
        max_tokens=4, temperature=0
    )
    print(pred)

Judge the following statements if they are likely to occur, only answer 'True' or 'False':
Statement: PersonX be real, thus as an result, PersonY argue PersonZ about it
Answer: True

Statement: PersonX come to talk to PersonZ, thus as an result, PersonY become enraged
Answer: False

Statement: PersonX argue with PersonY, thus as an result, PersonY be not a PersonZ
Answer: 
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " False"
    }
  ],
  "created": 1685531444,
  "id": "cmpl-7ME0yPVAUcRvwZJHqGbeEJjxGIwuY",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 1,
    "prompt_tokens": 91,
    "total_tokens": 92
  }
}
Judge the following statements if they are likely to occur, only answer 'True' or 'False':
Statement: PersonX come early, thus as an result, PersonX go home
Answer: True

Statement: PersonX go home early, thus as an result, PersonX come day
Answer: False

Statement:

### Least to Most

In [ ]:
# where to get these examples -> from dev set please!
exemplars = {
    "xReact": [
        [
            "PersonX prevent PersonY, thus as a result on PersonX's emotion, PersonX never reach out to anyone",
            "PersonX never reach out to anyone"
        ],
        [
            "PersonX go to sleep on hollow, thus as a result on PersonX's emotion, PersonX feel PersonX be tired",
            "PersonX feel PersonX be tired",
            "3. mental state",
            "PersonX go to sleep on hollow, thus as a result on PersonX's emotion, PersonX feel PersonX be tired"
        ],
        [
            "PersonX eat the sub, thus as a result on PersonX's emotion, PersonX feel PersonX be full",
            "PersonX feel PersonX be full",
            "3. mental state",
            "PersonX eat the sub, thus as a result on PersonX's emotion, PersonX feel PersonX be full"
        ]
    ],
    "oReact": [
        [
            "PersonX bear PersonY, thus as a result on PersonY's emotion, PersonY feel PersonY be here",
            "PersonY be here"
        ],
        [
            "PersonX play ana, thus as a result on PersonY's emotion, PersonY feel PersonY be fun",
            "PersonY be fun",
            "3. mental state",
            "PersonX play ana, thus as a result on PersonY's emotion, PersonY feel PersonY be fun"
        ],
        [
            "PersonX gaze at PersonY, thus as a result on PersonY's emotion, PersonY blush",
            "PersonY blush",
            "3. mental state",
            "PersonX gaze at PersonY, thus as a result on PersonY's emotion, PersonY blush"
        ]
    ],
    "xAttr": [
        [
            "PersonX hand slip, thus it can be seen about PersonX's attribute that PersonX fall",
            "PersonX fall"
        ],
        [
            "PersonX have not hear a car drive up, thus it can be seen about PersonX's attribute that PersonX be startle",
            "PersonX be startle",
            "3. mental state",
            "PersonX have not hear a car drive up, thus it can be seen about PersonX's attribute that PersonX be startle"
        ],
        [
            "PersonX walk PersonY through process, thus it can be seen about PersonX's attribute that PersonX be professional",
            "PersonX be professional",
            "2. persona",
            "PersonX walk PersonY through process, thus it can be seen about PersonX's attribute that PersonX be professional"
        ]
    ],
    "xIntent": [
        [
            "PersonX ride the light rail, thus it can be seen about PersonX's intention that PersonX do not need a car",
            "PersonX do not need a car",
            "PersonX ride the light rail",
            "PersonX do not need a car",
            "PersonX ride the light rail"
        ],
        [
            "PersonX would at have one person, thus it can be seen about PersonX's intention that PersonX wish",
            "PersonX wish",
            "PersonX would at have one person",
            "PersonX wish",
            "PersonX would at have one person",
            "Not determinable",
            "PersonX would at have one person, thus it can be seen about PersonX's intention that PersonX wish"
        ],
        [
            "PersonX can form a opinion, thus it can be seen about PersonX's intention that PersonX have a opinion",
            "PersonX have a opinion",
            "PersonX can form a opinion",
            "PersonX have a opinion",
            "PersonX can form a opinion",
            "0",
            "PersonX can form a opinion, thus it can be seen about PersonX's intention that PersonX have a opinion"
        ]
    ],
    "xNeed": [
        [
            "The event PersonX just activate it will not happen unless PersonX use it",
            "PersonX use it",
            "PersonX just activate it",
            "PersonX use it",
            "PersonX just activate it"
        ],
        [
            "The event PersonX go lady at counter will not happen unless PersonX be intolerant",
            "PersonX be intolerant",
            "PersonX go lady at counter",
            "PersonX be intolerant",
            "PersonX go lady at counter",
            "Not determinable",
            "The event PersonX go lady at counter will not happen unless PersonX be intolerant"
        ],
        [
            "The event PersonX pay by card will not happen unless PersonX can use PersonX debit card",
            "PersonX can use PersonX debit card",
            "PersonX pay by card",
            "PersonX can use PersonX debit card",
            "PersonX pay by card",
            "0",
            "The event PersonX pay by card will not happen unless PersonX can use PersonX debit card"
        ]
    ]
}

# import pandas as pd
# df = pd.read_csv('prompting_data/dev_data_to_tune.csv')
# a2atq = {k: v for k,v in zip(df['assertion'], df['assertion_tq'])}
# for r in exemplars.keys():
#     temp = exemplars[r]
#     for i in range(3):
#         for j in range(len(temp[i])):
#             temp[i][j] = a2atq.get(temp[i][j], temp[i][j])
#
# import json
# print(json.dumps(exemplars, indent=' '*4))

exemplars_tq = {
    "xReact": [
        [
            "If PersonX prevent PersonY, as a result, PersonX feels, PersonX never reach out to anyone",
            "PersonX never reach out to anyone"
        ],
        [
            "If PersonX go to sleep on hollow, as a result, PersonX feels, PersonX be tired",
            "PersonX feel PersonX be tired",
            "3. mental state",
            "If PersonX go to sleep on hollow, as a result, PersonX feels, PersonX be tired"
        ],
        [
            "If PersonX eat the sub, as a result, PersonX feels, PersonX be full",
            "PersonX feel PersonX be full",
            "3. mental state",
            "If PersonX eat the sub, as a result, PersonX feels, PersonX be full"
        ]
    ],
    "oReact": [
        [
            "If PersonX bear PersonY, as a result, PersonY or others feel, PersonY be here",
            "PersonY be here"
        ],
        [
            "If PersonX play ana, as a result, PersonY or others feel, PersonY be fun",
            "PersonY be fun",
            "3. mental state",
            "If PersonX play ana, as a result, PersonY or others feel, PersonY be fun"
        ],
        [
            "If PersonX gaze at PersonY, as a result, PersonY or others feel, PersonY blush",
            "PersonY blush",
            "3. mental state",
            "If PersonX gaze at PersonY, as a result, PersonY or others feel, PersonY blush"
        ]
    ],
    "xAttr": [
        [
            "If PersonX hand slip, PersonX is seen as, PersonX fall",
            "PersonX fall"
        ],
        [
            "If PersonX have not hear a car drive up, PersonX is seen as, PersonX be startle",
            "PersonX be startle",
            "3. mental state",
            "If PersonX have not hear a car drive up, PersonX is seen as, PersonX be startle"
        ],
        [
            "If PersonX walk PersonY through process, PersonX is seen as, PersonX be professional",
            "PersonX be professional",
            "2. persona",
            "If PersonX walk PersonY through process, PersonX is seen as, PersonX be professional"
        ]
    ],
    "xIntent": [
        [
            "If PersonX ride the light rail, because PersonX wanted, PersonX do not need a car",
            "PersonX do not need a car",
            "PersonX ride the light rail",
            "PersonX do not need a car",
            "PersonX ride the light rail"
        ],
        [
            "If PersonX would at have one person, because PersonX wanted, PersonX wish",
            "PersonX wish",
            "PersonX would at have one person",
            "PersonX wish",
            "PersonX would at have one person",
            "Not determinable",
            "If PersonX would at have one person, because PersonX wanted, PersonX wish"
        ],
        [
            "If PersonX can form a opinion, because PersonX wanted, PersonX have a opinion",
            "PersonX have a opinion",
            "PersonX can form a opinion",
            "PersonX have a opinion",
            "PersonX can form a opinion",
            "0",
            "If PersonX can form a opinion, because PersonX wanted, PersonX have a opinion"
        ]
    ],
    "xNeed": [
        [
            "If PersonX just activate it, but before, PersonX needed, PersonX use it",
            "PersonX use it",
            "PersonX just activate it",
            "PersonX use it",
            "PersonX just activate it"
        ],
        [
            "If PersonX go lady at counter, but before, PersonX needed, PersonX be intolerant",
            "PersonX be intolerant",
            "PersonX go lady at counter",
            "PersonX be intolerant",
            "PersonX go lady at counter",
            "Not determinable",
            "If PersonX go lady at counter, but before, PersonX needed, PersonX be intolerant"
        ],
        [
            "If PersonX pay by card, but before, PersonX needed, PersonX can use PersonX debit card",
            "PersonX can use PersonX debit card",
            "PersonX pay by card",
            "PersonX can use PersonX debit card",
            "PersonX pay by card",
            "0",
            "If PersonX pay by card, but before, PersonX needed, PersonX can use PersonX debit card"
        ]
    ]
}

In [ ]:
import copy
least2most_head_prompt = "For each statement below, please answer several questions to reach the final conclusion if the statement is commonsense." + \
" Whenever your answer of a question is No, please conclude that the statement is not commonsense. Otherwise, please conclude that the statement is commonsense.\n"

# assertion + prompt1
question_lists = {
    'typing': ["""
Statement: {}
Q: Which aspect (among three options 1. event/activity, 2. persona, 3. mental state) of the subject does the clause '{}' express. Answer the choice only.
A: """, """
Q: Is the above answer different from option 1. event/activity?
A: """, ''],
    'temporal': ["""
Statement: {}
Q: Which one of the following two related statements is more plausible:
0. {} before {}
1. {} after {}
Answer 0 or 1 only.
A: """, """
Q: Is the above answer different from 1?
A: """, '']
}
question_lists_tq = copy.deepcopy(question_lists)

question_lists['typing'][-1] = question_lists['temporal'][-1] = """
Q: Is the statement "{}" likely to occur?
A: """
question_lists_tq['typing'][-1] = question_lists_tq['temporal'][-1] = """
Q: Is the statement "{}" plausible?
A: """


def exemplar_prompt(e, question_lists):
    if e == 'typing':
        return  question_lists['typing'][0] + '1. event/activity' + \
                question_lists['typing'][1] + 'No. Thus, the statement is not commonsense\n' + \
                question_lists['typing'][0] + '{}' + \
                question_lists['typing'][1] + 'Yes' + \
                question_lists['typing'][2] + 'No. Thus, the statement is not commonsense\n' + \
                question_lists['typing'][0] + '{}' + \
                question_lists['typing'][1] + 'Yes' + \
                question_lists['typing'][2] + 'Yes. Thus, the statement is commonsense\n'
    elif e == 'temporal':
        return  question_lists['temporal'][0] + '1' + \
                question_lists['temporal'][1] + 'No. Thus, the statement is not commonsense\n' + \
                question_lists['temporal'][0] + '{}' + \
                question_lists['temporal'][1] + 'Yes' + \
                question_lists['temporal'][2] + 'No. Thus, the statement is not commonsense\n' + \
                question_lists['temporal'][0] + '{}' + \
                question_lists['temporal'][1] + 'Yes' + \
                question_lists['temporal'][2] + 'Yes. Thus, the statement is commonsense\n'


e = 'typing'
r = 'xReact'
# least2most_prompt1
print(exemplar_prompt(e, question_lists_tq).format(*[c for instance in exemplars_tq[r] for c in instance]))
# least2most_prompt2
print(exemplar_prompt(e, question_lists_tq).format(*[c for instance in exemplars[r] for c in instance]))
# least2most_prompt3
print(exemplar_prompt(e, question_lists).format(*[c for instance in exemplars[r] for c in instance]))



Statement: If PersonX prevent PersonY, as a result, PersonX feels, PersonX never reach out to anyone
Q: Which aspect (among three options 1. event/activity, 2. persona, 3. mental state) of the subject does the clause 'PersonX never reach out to anyone' express. Answer the choice only.
A: 1. event/activity
Q: Is the above answer different from option 1. event/activity?
A: No. Thus, the statement is not commonsense

Statement: If PersonX go to sleep on hollow, as a result, PersonX feels, PersonX be tired
Q: Which aspect (among three options 1. event/activity, 2. persona, 3. mental state) of the subject does the clause 'PersonX feel PersonX be tired' express. Answer the choice only.
A: 3. mental state
Q: Is the above answer different from option 1. event/activity?
A: Yes
Q: Is the statement "If PersonX go to sleep on hollow, as a result, PersonX feels, PersonX be tired" plausible?
A: No. Thus, the statement is not commonsense

Statement: If PersonX eat the sub, as a result, PersonX feels

In [ ]:
data_df['prediction'] = 0

for r in ['xReact', 'oReact', 'xAttr', 'xIntent', 'xNeed']:
    e = 'typing' if r in ['xReact', 'oReact', 'xAttr'] else 'temporal'
    arguments = [c for instance in exemplars[r] for c in instance]
    constant_prompt = least2most_head_prompt + exemplar_prompt(e, question_lists).format(*arguments)
    # print(constant_prompt); continue

    for i in data_df.index[data_df['relation'] == r]:
        tail = data_df['tail'][i]
        head = data_df['head'][i]
        assertion = data_df['assertion'][i]
        if e == 'typing':
            instance_prompt = question_lists[e][0].format(assertion, tail)
        else:
            instance_prompt = question_lists[e][0].format(assertion, tail, head, tail, head)

        # stage 1
        answer = chatgpt(constant_prompt + instance_prompt, 5)
        instance_prompt += answer + question_lists[e][1]

        # stage 2
        answer = chatgpt(constant_prompt + instance_prompt, 2)
        instance_prompt += answer
        # if 'no' in answer.lower():
        #     print(instance_prompt)
        #     continue

        # stage 3
        instance_prompt += question_lists[e][2].format(assertion)
        answer = chatgpt(constant_prompt + instance_prompt, 10)
        instance_prompt += answer
        if 'yes' in answer:
            data_df['prediction'][i] = 1

        print(instance_prompt)
        break

In [ ]:
# no exemplar
data_df['prediction'] = 0
func = chatgpt

for r in ['xReact', 'oReact', 'xAttr', 'xIntent', 'xNeed']:
    e = 'typing' if r in ['xReact', 'oReact', 'xAttr'] else 'temporal'
    arguments = [c for instance in exemplars[r] for c in instance]
    constant_prompt = least2most_head_prompt #+ exemplar_prompt(e, question_lists).format(*arguments)
    # print(constant_prompt); continue

    for i in data_df.index[data_df['relation'] == r]:
        tail = data_df['tail'][i]
        head = data_df['head'][i]
        assertion = data_df['assertion'][i]
        if e == 'typing':
            instance_prompt = question_lists[e][0].format(assertion, tail)
        else:
            instance_prompt = question_lists[e][0].format(assertion, tail, head, tail, head)

        # stage 1
        answer = func(constant_prompt + instance_prompt, 5)
        instance_prompt += answer + question_lists[e][1]

        # stage 2
        answer = func(constant_prompt + instance_prompt, 2)
        instance_prompt += answer
        # if 'no' in answer.lower():
        #     print(instance_prompt)
        #     continue

        # stage 3
        instance_prompt += question_lists[e][2].format(assertion)
        answer = func(constant_prompt + instance_prompt, 10)
        instance_prompt += answer
        if 'yes' in answer:
            data_df['prediction'][i] = 1

        print(instance_prompt)
        break


Statement: PersonX pull off, thus as a result on PersonX's emotion, PersonX wait
Q: Which aspect (among three options 1. event/activity, 2. persona, 3. mental state) of the subject does the clause 'PersonX wait' express. Answer the choice only.
A: 3. mental state
Q: Is the above answer different from option 1. event/activity?
A: Yes
Q: Is the statement "PersonX pull off, thus as a result on PersonX's emotion, PersonX wait" likely to occur?
A: No

Statement: PersonX bear PersonY, thus as a result on PersonY's emotion, PersonY feel PersonY be here
Q: Which aspect (among three options 1. event/activity, 2. persona, 3. mental state) of the subject does the clause 'PersonY be here' express. Answer the choice only.
A: 3. mental state
Q: Is the above answer different from option 1. event/activity?
A: Yes,
Q: Is the statement "PersonX bear PersonY, thus as a result on PersonY's emotion, PersonY feel PersonY be here" likely to occur?
A: No, the statement does not make sense and is

Statement: 

## text-davinci-003

Test if text-davinci-003 is a strict model?

#### direct

In [ ]:
predictions = []
bs = 20

for i in range((len(data)-1)//bs + 1):
    p = openai.Completion.create(
        model="text-davinci-003",
        prompt=[judge_prompt_1.format(x) for x in data[i*bs:(i+1)*bs]],
        max_tokens=20,
        temperature=0
    )
    print(p['choices'])
    predictions.extend(p['choices'])

# omg, only 5 True!

In [ ]:
judge_prompt_4 = "Judge the following statements if they are likely to occur, only list question number and answer 'True' or 'False' for each:\n{}"
bs = 20

for i in range(1, (len(data)-1)//bs + 1):
    pred = openai.Completion.create(
        model="text-davinci-003",
        prompt=judge_prompt_4.format('\n'.join([f'{j}. {x}' for j, x in enumerate(data[i*bs:(i+1)*bs])])),
        max_tokens=500,
        temperature=0
    )
    print(pred)

# not so bad 0.7 auc, but :))

batch judge is .. better than individual judge, as individual judge is too strict (only 5 out of 102 are True)

#### CoT

In [ ]:
p = openai.Completion.create(
    model="text-davinci-003",
    prompt=[zeroshotCoT_prompt_tq.format(data_df['assertion'][s])
        for s in [0, 4] # range(i*bs, min((i+1)*bs, len(data_df)))
    ],
    max_tokens=50,
    temperature=0
)
temp = '\n'.join([x['text'].replace('\n','--') for x in p['choices']])
print(temp)

----False. Ejaculation does not necessarily mean that a person wants to experience an orgasm. It is possible for a person to ejaculate without wanting or experiencing an orgasm.
----True. Enlisting in the military is a sign of wanting to fight for one's country.


In [ ]:
p = openai.Completion.create(
    model="text-davinci-003",
    prompt=[zeroshotCoT_prompt1.format(data_df['assertion_tq'][s])
        for s in [0, 4] # range(i*bs, min((i+1)*bs, len(data_df)))
    ],
    max_tokens=50,
    temperature=0
)
temp = '\n'.join([x['text'].replace('\n','--') for x in p['choices']])
print(temp)

----True. Ejaculation often leads to orgasm, so it is likely to occur.
----Step 1: PersonX enlists in the military.--True----Step 2: As a result, PersonX wants to fight for their country.--True----Conclusion: True


In [ ]:
p = openai.Completion.create(
    model="text-davinci-003",
    prompt=[zeroshotCoT_prompt1.format(data_df['assertion'][s])
        for s in range(20) # range(i*bs, min((i+1)*bs, len(data_df)))
    ],
    max_tokens=50,
    temperature=0
)
temp = '\n'.join([x['text'].replace('\n','--') for x in p['choices']])
print(temp)

----False. Ejaculation does not necessarily mean that a person wants to experience an orgasm. It is possible for a person to ejaculate without wanting or experiencing an orgasm.
----False. PersonX may go to the fridge for many reasons, such as to get a snack or a drink of water.
----False. Working tirelessly does not necessarily mean that a person wants to die.
----True
----Step 1: Is PersonX enlisting?--Yes----Step 2: Does PersonX want to fight for their country?--Yes----Conclusion: True
----False. It is unlikely that a person falling into a pool would lead to their mother enrolling them in school.
----False. It is unlikely that a person would want to put down toilet paper in the toilet after they have already defecated.
----False. It is unlikely that a person would come and want to be paranoid.
----False. This statement does not make sense.
----False. It is unlikely that PersonX would give PersonY a substantial amount if they knew PersonY could not pay it back.
----True
----False. Ha

#### abstraction

In [ ]:
pred = openai.Completion.create(
    model="text-davinci-003",
    prompt=old_conceptualization_prompt.format(data[34], 'emotion'),
    max_tokens=100,
    temperature=0
)

pred

In [ ]:
def query_w_abstraction(idx, conceptualization_prompt=conceptualization_prompt):
    if type(idx) != list:
        idx = [idx]
    pred = openai.Completion.create(
        model="text-davinci-003",
        prompt=[conceptualization_prompt.format(
            data_df['assertion'][i],
            data_df['dim'][i],
            data_df['head'][i],
            data_df['tail'][i]
        ) for i in idx],
        max_tokens=100,
        temperature=0
    )

    print(pred)

##### all dev instances

same-prompt batch (less effective than individual)

individual

5 exemplars: actually better @@

In [ ]:
predictions = []
bs = 10

for b in range((len(data)-1)//bs + 1):
    p = openai.Completion.create(
        model="text-davinci-003",
        prompt=[conceptualization_prompt_5exemplars.format(
            data_df['assertion'][i],
            data_df['dim'][i],
            data_df['head'][i],
            data_df['tail'][i])
            for i in range(b*bs, min((b+1)*bs, len(data_df)))
        ],
        max_tokens=100,
        temperature=0
    )
    print(p['choices'])
    predictions.extend(p['choices'])

In [ ]:
print('\n'.join('##' + str(x['index']) + x['text'] for x in predictions))
# print('\n'.join(str(x['index']) + x['text'].rsplit('\n',1)[1][1:] for x in predictions))

two exemplars

In [ ]:
predictions = []
bs = 10

for b in range((len(data)-1)//bs + 1):
    p = openai.Completion.create(
        model="text-davinci-003",
        prompt=[conceptualization_prompt.format(
            data_df['assertion'][i],
            data_df['dim'][i],
            data_df['head'][i],
            data_df['tail'][i])
            for i in range(b*bs, min((b+1)*bs, len(data_df)))
        ],
        max_tokens=100,
        temperature=0
    )
    print(p['choices'])
    predictions.extend(p['choices'])

In [ ]:
print('\n'.join('##' + str(x['index']) + x['text'] for x in predictions))
# print('\n'.join(str(x['index']) + x['text'].rsplit('\n',1)[1][1:] for x in predictions))

##### draft experiment

batch

- run slower
- yield longer answer (repeat instance) -> cost more
- less accurate?

In [ ]:
samples = [conceptualization_prompt_batch_input_tail.format(
            data_df['assertion'][i],
            data_df['dim'][i],
            data_df['head'][i],
            data_df['tail'][i]
            ) for i in [34, 38, 68, 69]]
prompt = conceptualization_prompt_batch_input + '\n'.join(samples)
    # '\n'.join([f'{j}. {x}' for j, x in enumerate(samples)])
print(prompt)

In [ ]:
pred = openai.Completion.create(
    model="text-davinci-003",
    prompt=prompt,
    max_tokens=600,
    temperature=0
)
print(pred)
print(pred['choices'][0]['text'])

individual

In [ ]:
for i in [34, 38, 68, 69]:
    query_w_abstraction(i)

Very unpredictable! more exemplar, worse judgment!

In [ ]:
# use conceptualization_prompt_5exemplars

for i in [34, 38, 68, 69]:
    query_w_abstraction(i, conceptualization_prompt_5exemplars)

In [ ]:
query_w_abstraction(69)

Insight: text-davinci-003 judging samples individually seems to be strict

### code-davinci-003

OMG, no code-davinci-003, while code-davinci-002 is deprecated

## OpenAI official request batching

In [ ]:
prompt = judge_prompt_1.format(data[0])
gen_chatgpt = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0301",
    messages=[{"role": "user", "content": prompt}],
    max_tokens=7, temperature=0
)

In [ ]:
gen_chatgpt

In [ ]:
gen_chatgpt = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0301",
    messages=[[{"role": "user", "content": judge_prompt_1.format(s)}] for s in data[1:4]],
    max_tokens=4, temperature=0
)

In [ ]:
openai.ChatCompletion.create.__doc__
# ChatCompletion doesn't support batching yet!

In [ ]:
num_stories = 2
prompts = ["Once upon a time,"] * num_stories

# batched example, with 10 story completions per request
response = openai.Completion.create(
    model="curie",
    prompt=prompts,
    max_tokens=20,
)

In [ ]:
response

## [IMPORTANT] Run on test set

In [ ]:
tst_data_df = pd.read_csv('data/tst_data_to_eval.csv')
tst_data = tst_data_df['assertion'].tolist()

tst_kate_exemplars = pd.read_csv('data/tst_kate_exemplars_top5.csv')
tst_kate_exemplars_tq = pd.read_csv('data/tst_kate_exemplars_tq_top5.csv')
tst_kate_same_relation_exemplars = pd.read_csv('data/tst_kate_same_relation_exemplars_top5.csv')
tst_kate_same_relation_exemplars_tq = pd.read_csv('data/tst_kate_same_relation_exemplars_tq_top5.csv')
tst_random_exemplars = pd.read_csv('data/tst_random_5exemplars.csv')

print(tst_data_df.columns, len(tst_data), len(tst_kate_exemplars))

Index(['head', 'relation', 'tail', 'class', 'split', 'label', 'dim',
       'assertion', 'assertion_tq', 'naive1', 'naive2', 'naive3'],
      dtype='object') 979 979


In [ ]:
for df in [tst_kate_exemplars,
  tst_kate_exemplars_tq,
  tst_kate_same_relation_exemplars,
  tst_kate_same_relation_exemplars_tq,
  tst_random_exemplars]:
  print(df.columns)

Index(['0', '0_label', '1', '1_label', '2', '2_label', '3', '3_label', '4',
       '4_label'],
      dtype='object')
Index(['0_tq', '0_tq_label', '1_tq', '1_tq_label', '2_tq', '2_tq_label',
       '3_tq', '3_tq_label', '4_tq', '4_tq_label'],
      dtype='object')
Index(['0', '0_label', '1', '1_label', '2', '2_label', '3', '3_label', '4',
       '4_label'],
      dtype='object')
Index(['0_tq', '0_tq_label', '1_tq', '1_tq_label', '2_tq', '2_tq_label',
       '3_tq', '3_tq_label', '4_tq', '4_tq_label'],
      dtype='object')
Index(['0', '0_label', '1', '1_label', '2', '2_label', '3', '3_label', '4',
       '4_label', '0_tq', '0_tq_label', '1_tq', '1_tq_label', '2_tq',
       '2_tq_label', '3_tq', '3_tq_label', '4_tq', '4_tq_label', '5',
       '5_label', '6', '6_label', '7', '7_label', '8', '8_label', '9',
       '9_label'],
      dtype='object')


In [ ]:
predictions = []
# output = open('data/output.txt', 'a+')
# output.close()
# openai.api_key =

In [ ]:
bs = 5 #@param
total = (len(tst_data_df)-1)//bs+1
timeout_succeed = 1 #@param
timeout_failure = 10 #@param

In [ ]:
async def chatgpt_loop(customize_prompt, max_tokens, filename_suffix, resume_at_step=0, end_at_step=total):
    for i in range(resume_at_step, end_at_step):
        while True:
            try:
                print(i, '/', total)
                pred = await dispatch_openai_requests(
                        messages_list = [[{"role": "user", "content": customize_prompt(s)}]
                            for s in range(i*bs, min((i+1)*bs, len(tst_data_df)))],
                        max_tokens=max_tokens,
                    )
                temp = ', '.join([x['choices'][0]['message']['content'] for x in pred])
                print(temp)
                output = open(f'data/tst_gpt3.5-turbo_{filename_suffix}.txt', 'a+')
                output.write(f'{i}/{total}\n{temp}\n')
                output.close()
                predictions.extend([x['choices'][0]['message']['content'] for x in pred])
                time.sleep(timeout_succeed)
                break
            except Exception as e:
                print(e)
                time.sleep(timeout_failure)


In [ ]:
def davinci_loop(customize_prompt, max_tokens, filename_suffix, resume_at_step=0, end_at_step=total):
    for i in range(resume_at_step, end_at_step):
        while True:
            try:
                print(i, '/', total)
                p = openai.Completion.create(
                    model="text-davinci-003",
                    prompt=[customize_prompt(s)
                        for s in range(i*bs, min((i+1)*bs, len(tst_data_df)))],
                    max_tokens=max_tokens,
                    temperature=0
                )
                temp = '\n'.join([x['text'].replace('\n','--') for x in p['choices']])
                print(temp)
                output = open(f'data/tst_davinci_{filename_suffix}.txt', 'a+')
                output.write(f'{i}/{total}\n{temp}\n')
                output.close()
                predictions.extend([x['text'].replace('\n','--') for x in p['choices']])
                time.sleep(timeout_succeed)
                break
            except Exception as e:
                print(e)
                time.sleep(timeout_failure)

### gpt3.5-turbo

naive

In [ ]:
naive_prompt1 = lambda s:judge_prompt_0.format(tst_data_df['assertion_tq'][s])
print(naive_prompt1(0))

await chatgpt_loop(naive_prompt1, max_tokens=2, filename_suffix='naive_prompt1', resume_at_step=0)

In [ ]:
naive_prompt2 = lambda s:judge_prompt_0.format(tst_data_df['assertion'][s])
print(naive_prompt2(0))

await chatgpt_loop(naive_prompt2, max_tokens=2, filename_suffix='naive_prompt2', resume_at_step=0)

In [ ]:
naive_prompt3 = lambda s:judge_prompt_1.format(tst_data_df['assertion'][s])
print(naive_prompt3(0))

await chatgpt_loop(naive_prompt3, max_tokens=2, filename_suffix='naive_prompt3', resume_at_step=0)

moe ~ system1

In [ ]:
# run MoE first
for i in range(0, total):
    while True:
        try:
            print(i, '/', total)
            pred = await dispatch_openai_requests(
                    messages_list = [[{"role": "user", "content": moe_q1_prompt1.format(s)}] for s in tst_data_df['tail'][i*bs:(i+1)*bs]],
                    max_tokens=4,
                )
            temp = ', '.join([x['choices'][0]['message']['content'] for x in pred])
            print(temp)
            output = open('data/output.txt', 'a+')
            output.write(f'{i}/{total}\n{temp}\n')
            output.close()
            predictions.extend([x['choices'][0]['message']['content'] for x in pred])
            time.sleep(timeout_succeed)
            break
        except Exception as e:
            print(e)
            time.sleep(timeout_failure)


In [ ]:
# run MoE q2head
for i in range(0, total):
    while True:
        try:
            print(i, '/', total)
            pred = await dispatch_openai_requests(
                    messages_list = [[{"role": "user", "content": moe_q2_prompt1.format(s)}] for s in tst_data_df['head'][i*bs:(i+1)*bs]],
                    max_tokens=4,
                )
            temp = ', '.join([x['choices'][0]['message']['content'] for x in pred])
            print(temp)
            output = open('data/tst_gpt3.5-turbo_moe_q2head_prompt1.txt', 'a+')
            output.write(f'{i}/{total}\n{temp}\n')
            output.close()
            predictions.extend([x['choices'][0]['message']['content'] for x in pred])
            time.sleep(timeout_succeed)
            break
        except Exception as e:
            print(e)
            time.sleep(timeout_failure)


In [ ]:
# run MoE q2tail
for i in range(0, total):
    while True:
        try:
            print(i, '/', total)
            pred = await dispatch_openai_requests(
                    messages_list = [[{"role": "user", "content": moe_q2_prompt1.format(s)}] for s in tst_data_df['tail'][i*bs:(i+1)*bs]],
                    max_tokens=4,
                )
            temp = ', '.join([x['choices'][0]['message']['content'] for x in pred])
            print(temp)
            output = open('data/tst_gpt3.5-turbo_moe_q2tail_prompt1.txt', 'a+')
            output.write(f'{i}/{total}\n{temp}\n')
            output.close()
            predictions.extend([x['choices'][0]['message']['content'] for x in pred])
            time.sleep(timeout_succeed)
            break
        except Exception as e:
            print(e)
            time.sleep(timeout_failure)


In [ ]:
moe_q2_prompt2_fn = lambda s:moe_q2_prompt2.format(tst_data_df['head'][s], tst_data_df['tail'][s])
print(moe_q2_prompt2_fn(0))

await chatgpt_loop(moe_q2_prompt2_fn, max_tokens=2, filename_suffix='moe_q2_prompt2', resume_at_step=0)

In [ ]:
idx_q3 = tst_data_df['relation'].isin(['xIntent', 'xNeed'])
temp_df = tst_data_df[idx_q3]
temp_df.reset_index(inplace=True)
total = (len(temp_df)-1)//bs+1
temp_df

for b in range(0, total):
    while True:
        try:
            print(b, '/', total)
            pred = await dispatch_openai_requests(
                    messages_list = [[{"role": "user", "content": moe_q3_prompt3.format(temp_df['tail'][i], temp_df['head'][i])}] \
                        for i in range(b*bs, min((b+1)*bs, len(temp_df)))],
                    max_tokens=4,
                )
            temp = ', '.join([x['choices'][0]['message']['content'] for x in pred])
            print(temp)
            output = open('data/tst_gpt3.5-turbo_moe_q3_prompt3.txt', 'a+')
            output.write(f'{b}/{total}\n{temp}\n')
            output.close()
            predictions.extend([x['choices'][0]['message']['content'] for x in pred])
            time.sleep(timeout_succeed)
            break
        except Exception as e:
            print(e)
            time.sleep(timeout_failure)


In [ ]:
for b in range(0, total):
    while True:
        try:
            print(b, '/', total)
            pred = await dispatch_openai_requests(
                    messages_list = [[{"role": "user", "content": moe_q3_prompt3.format(tst_data_df['tail'][i], tst_data_df['head'][i])}] \
                        for i in range(b*bs, min((b+1)*bs, len(tst_data_df)))],
                    max_tokens=2,
                )
            temp = ', '.join([x['choices'][0]['message']['content'] for x in pred])
            print(temp)
            output = open('data/tst_gpt3.5-turbo_moe_q3_prompt3.txt', 'a+')
            output.write(f'{b}/{total}\n{temp}\n')
            output.close()
            predictions.extend([x['choices'][0]['message']['content'] for x in pred])
            time.sleep(timeout_succeed)
            break
        except Exception as e:
            print(e)
            time.sleep(timeout_failure)


In [ ]:
temp_df = tst_data_df[tst_data_df['relation'].isin('xReact,oReact,xAttr'.split(','))]
temp_df.reset_index(inplace=True)
total = (len(temp_df)-1)//bs+1
temp_df

for b in range(0, total):
    while True:
        try:
            print(b, '/', total)
            pred = await dispatch_openai_requests(
                    messages_list = [[{"role": "user", "content": moe_q1_prompt2.format(temp_df['tail'][i])}] \
                        for i in range(b*bs, min((b+1)*bs, len(temp_df)))],
                    max_tokens=4,
                )
            temp = ', '.join([x['choices'][0]['message']['content'] for x in pred])
            print(temp)
            output = open('data/tst_gpt3.5-turbo_moe_q1_prompt2.txt', 'a+')
            output.write(f'{b}/{total}\n{temp}\n')
            output.close()
            predictions.extend([x['choices'][0]['message']['content'] for x in pred])
            time.sleep(timeout_succeed)
            break
        except Exception as e:
            print(e)
            time.sleep(timeout_failure)


In [ ]:
temp_df = tst_data_df[tst_data_df['relation'].isin(['xIntent', 'xNeed'])]
temp_df.reset_index(inplace=True)
total = (len(temp_df)-1)//bs+1
temp_df

for b in range(0, total):
    while True:
        try:
            print(b, '/', total)
            pred = await dispatch_openai_requests(
                    messages_list = [[{"role": "user", "content": moe_q3_prompt1.format(temp_df['tail'][i], temp_df['head'][i])}] \
                        for i in range(b*bs, min((b+1)*bs, len(temp_df)))],
                    max_tokens=4,
                )
            temp = ', '.join([x['choices'][0]['message']['content'] for x in pred])
            print(temp)
            output = open('data/tst_gpt3.5-turbo_moe_q3_prompt1.txt', 'a+')
            output.write(f'{b}/{total}\n{temp}\n')
            output.close()
            predictions.extend([x['choices'][0]['message']['content'] for x in pred])
            time.sleep(timeout_succeed)
            break
        except Exception as e:
            print(e)
            time.sleep(timeout_failure)


zeroshotCoT

In [ ]:
for i in range(0, total):
    while True:
        try:
            print(i, '/', total)
            pred = await dispatch_openai_requests(
                    messages_list = [[{"role": "user", "content": zeroshotCoT_prompt_tq.format(tst_data_df['assertion'][s])}]
                        for s in range(i*bs, min((i+1)*bs, len(tst_data_df)))],
                    max_tokens=150,
                )
            temp = '\n--'.join([x['choices'][0]['message']['content'] for x in pred])
            print(temp)
            output = open('data/tst_gpt3.5-turbo_zeroshot_prompt1.txt', 'a+')
            output.write(f'{i}/{total}\n{temp}\n')
            output.close()
            predictions.extend([x['choices'][0]['message']['content'] for x in pred])
            time.sleep(timeout_succeed)
            break
        except Exception as e:
            print(e)
            time.sleep(timeout_failure)


In [ ]:
for i in range(0, total):
    while True:
        try:
            print(i, '/', total)
            pred = await dispatch_openai_requests(
                    messages_list = [[{"role": "user", "content": zeroshotCoT_prompt1.format(tst_data_df['assertion_tq'][s])}]
                        for s in range(i*bs, min((i+1)*bs, len(tst_data_df)))],
                    max_tokens=150,
                )
            temp = '\n--'.join([x['choices'][0]['message']['content'] for x in pred])
            print(temp)
            output = open('data/tst_gpt3.5-turbo_zeroshot_prompt2.txt', 'a+')
            output.write(f'{i}/{total}\n{temp}\n')
            output.close()
            predictions.extend([x['choices'][0]['message']['content'] for x in pred])
            time.sleep(timeout_succeed)
            break
        except Exception as e:
            print(e)
            time.sleep(timeout_failure)


In [ ]:
for i in range(0, total):
    while True:
        try:
            print(i, '/', total)
            pred = await dispatch_openai_requests(
                    messages_list = [[{"role": "user", "content": zeroshotCoT_prompt1.format(tst_data_df['assertion'][s])}]
                        for s in range(i*bs, min((i+1)*bs, len(tst_data_df)))],
                    max_tokens=150,
                )
            temp = '\n--'.join([x['choices'][0]['message']['content'] for x in pred])
            print(temp)
            output = open('data/tst_gpt3.5-turbo_zeroshot_prompt3.txt', 'a+')
            output.write(f'{i}/{total}\n{temp}\n')
            output.close()
            predictions.extend([x['choices'][0]['message']['content'] for x in pred])
            time.sleep(timeout_succeed)
            break
        except Exception as e:
            print(e)
            time.sleep(timeout_failure)


kate top1

In [ ]:
# kate_prompt1
for i in range(0, total):
    while True:
        try:
            print(i, '/', total)
            pred = await dispatch_openai_requests(
                     messages_list = [
                        [{
                        "role": "user",
                        "content": kate_prompt_0.format('\n\n'.join([
                            'Statement: {}\nAnswer: {}'.format(tst_kate_exemplars['kate_pos_0_tq'][s], 'Yes'),
                            'Statement: {}\nAnswer: {}'.format(tst_kate_exemplars['kate_neg_0_tq'][s], 'No'),
                            'Statement: {}\nAnswer: {}'.format(tst_data_df['assertion_tq'][s], '')]))
                        }]
                        for s in range(i*bs, min((i+1)*bs, len(tst_data_df)))
                    ],
                    max_tokens=2,
                )
            temp = ', '.join([x['choices'][0]['message']['content'] for x in pred])
            print(temp)
            output = open('data/tst_gpt3.5-turbo_kate_top1_prompt1.txt', 'a+')
            output.write(f'{i}/{total}\n{temp}\n')
            output.close()
            predictions.extend([x['choices'][0]['message']['content'] for x in pred])
            time.sleep(timeout_succeed)
            break
        except Exception as e:
            print(e)
            time.sleep(timeout_failure)


In [ ]:
# kate_prompt2
for i in range(0 total):
    while True:
        try:
            print(i, '/', total)
            pred = await dispatch_openai_requests(
                     messages_list = [
                        [{
                        "role": "user",
                        "content": kate_prompt_0.format('\n\n'.join([
                            'Statement: {}\nAnswer: {}'.format(tst_kate_exemplars['kate_pos_0'][s], 'Yes'),
                            'Statement: {}\nAnswer: {}'.format(tst_kate_exemplars['kate_neg_0'][s], 'No'),
                            'Statement: {}\nAnswer: {}'.format(tst_data_df['assertion'][s], '')]))
                        }]
                        for s in range(i*bs, min((i+1)*bs, len(tst_data_df)))
                    ],
                    max_tokens=2,
                )
            temp = ', '.join([x['choices'][0]['message']['content'] for x in pred])
            print(temp)
            output = open('data/tst_gpt3.5-turbo_kate_top1_prompt2.txt', 'a+')
            output.write(f'{i}/{total}\n{temp}\n')
            output.close()
            predictions.extend([x['choices'][0]['message']['content'] for x in pred])
            time.sleep(timeout_succeed)
            break
        except Exception as e:
            print(e)
            time.sleep(timeout_failure)


In [ ]:
# kate_prompt3
for i in range(0, total):
    while True:
        try:
            print(i, '/', total)
            pred = await dispatch_openai_requests(
                     messages_list = [
                        [{
                        "role": "user",
                        "content": kate_prompt_1.format('\n\n'.join([
                            'Statement: {}\nAnswer: {}'.format(tst_kate_exemplars['kate_pos_0'][s], 'True'),
                            'Statement: {}\nAnswer: {}'.format(tst_kate_exemplars['kate_neg_0'][s], 'False'),
                            'Statement: {}\nAnswer: {}'.format(tst_data_df['assertion'][s], '')]))
                        }]
                        for s in range(i*bs, min((i+1)*bs, len(tst_data_df)))
                    ],
                    max_tokens=2,
                )
            temp = ', '.join([x['choices'][0]['message']['content'] for x in pred])
            print(temp)
            output = open('data/tst_gpt3.5-turbo_kate_top1_prompt3.txt', 'a+')
            output.write(f'{i}/{total}\n{temp}\n')
            output.close()
            predictions.extend([x['choices'][0]['message']['content'] for x in pred])
            time.sleep(timeout_succeed)
            break
        except Exception as e:
            print(e)
            time.sleep(timeout_failure)


kate top5

In [ ]:
kate_top5_prompt1 = lambda s:kate_prompt_0.format('\n\n'.join(
    ['Statement: {}\nAnswer: {}'.format(tst_kate_exemplars[f'kate_{p}_{_}_tq'][s], a) \
        for _ in range(5) for p, a in [('pos', 'Yes'), ('neg', 'No')]] +
    ['Statement: {}\nAnswer: {}'.format(tst_data_df['assertion_tq'][s], '')]
))
print(kate_top5_prompt1(0))

# await chatgpt_loop(kate_top5_prompt1, max_tokens=2, filename_suffix='kate_top5_prompt1', resume_at_step=0)

In [ ]:
kate_top5_prompt2 = lambda s:kate_prompt_0.format('\n\n'.join(
    ['Statement: {}\nAnswer: {}'.format(tst_kate_exemplars[f'kate_{p}_{_}'][s], a) \
        for _ in range(5) for p, a in [('pos', 'Yes'), ('neg', 'No')]] +
    ['Statement: {}\nAnswer: {}'.format(tst_data_df['assertion'][s], '')]
))
print(kate_top5_prompt2(0))

# await chatgpt_loop(kate_top5_prompt2, max_tokens=2, filename_suffix='kate_top5_prompt2', resume_at_step=0)

In [ ]:
kate_top5_prompt3 = lambda s:kate_prompt_1.format('\n\n'.join(
    ['Statement: {}\nAnswer: {}'.format(tst_kate_exemplars[f'kate_{p}_{_}'][s], a) \
        for _ in range(5) for p, a in [('pos', 'True'), ('neg', 'False')]] +
    ['Statement: {}\nAnswer: {}'.format(tst_data_df['assertion'][s], '')]
))
print(kate_top5_prompt3(0))

# await chatgpt_loop(kate_top5_prompt3, max_tokens=2, filename_suffix='kate_top5_prompt3', resume_at_step=0)

least to most

In [ ]:
tst_data_df['prediction'] = 0
func = chatgpt
run = 1
# need to control three variables: question_lists_tq, exemplars_tq, assertion_tq

for r in ['xReact', 'oReact', 'xAttr', 'xIntent', 'xNeed']:
    e = 'typing' if r in ['xReact', 'oReact', 'xAttr'] else 'temporal'
    arguments = [c for instance in exemplars_tq[r] for c in instance]
    constant_prompt = least2most_head_prompt + exemplar_prompt(e, question_lists_tq).format(*arguments)
    # print(constant_prompt); continue

    for i in tst_data_df.index[tst_data_df['relation'] == r]:
        tail = tst_data_df['tail'][i]
        head = tst_data_df['head'][i]
        assertion_tq = tst_data_df['assertion_tq'][i]
        if e == 'typing':
            instance_prompt = question_lists_tq[e][0].format(assertion_tq, tail)
        else:
            instance_prompt = question_lists_tq[e][0].format(assertion_tq, tail, head, tail, head)

        # stage 1
        answer = func(constant_prompt + instance_prompt, 5)
        instance_prompt += answer + question_lists_tq[e][1]

        # stage 2
        answer = func(constant_prompt + instance_prompt, 2)
        instance_prompt += answer
        if 'no' in answer.lower():
            print(f'--{i}\n{instance_prompt}\n')
            output = open(f'data/tst_gpt3.5-turbo_least2most_prompt{run}_full.txt', 'a+')
            output.write(f'--{i}\n{instance_prompt}\n')
            output.close()
            continue

        # stage 3
        instance_prompt += question_lists_tq[e][2].format(assertion_tq)
        answer = func(constant_prompt + instance_prompt, 10)
        instance_prompt += answer
        if 'yes' in answer.lower():
            tst_data_df['prediction'][i] = 1

        print(f'--{i}\n{instance_prompt}\n')
        output = open(f'data/tst_gpt3.5-turbo_least2most_prompt{run}_full.txt', 'a+')
        output.write(f'--{i}\n{instance_prompt}\n')
        output.close()

tst_data_df['prediction'].to_csv(f'data/tst_gpt3.5-turbo_least2most_prompt{run}.csv', index=False)

In [ ]:
tst_data_df['prediction'] = 0
func = chatgpt
run = 2
# need to control three variables: question_lists_tq, exemplars, assertion

for r in ['xReact', 'oReact', 'xAttr', 'xIntent', 'xNeed']:
    e = 'typing' if r in ['xReact', 'oReact', 'xAttr'] else 'temporal'
    arguments = [c for instance in exemplars[r] for c in instance]
    constant_prompt = least2most_head_prompt + exemplar_prompt(e, question_lists_tq).format(*arguments)
    # print(constant_prompt); continue

    for i in tst_data_df.index[tst_data_df['relation'] == r]:
        tail = tst_data_df['tail'][i]
        head = tst_data_df['head'][i]
        assertion = tst_data_df['assertion'][i]
        if e == 'typing':
            instance_prompt = question_lists_tq[e][0].format(assertion, tail)
        else:
            instance_prompt = question_lists_tq[e][0].format(assertion, tail, head, tail, head)

        # stage 1
        answer = func(constant_prompt + instance_prompt, 5)
        instance_prompt += answer + question_lists_tq[e][1]

        # stage 2
        answer = func(constant_prompt + instance_prompt, 2)
        instance_prompt += answer
        if 'no' in answer.lower():
            print(f'--{i}\n{instance_prompt}\n')
            output = open(f'data/tst_gpt3.5-turbo_least2most_prompt{run}_full.txt', 'a+')
            output.write(f'--{i}\n{instance_prompt}\n')
            output.close()
            continue

        # stage 3
        instance_prompt += question_lists_tq[e][2].format(assertion)
        answer = func(constant_prompt + instance_prompt, 10)
        instance_prompt += answer
        if 'yes' in answer.lower():
            tst_data_df['prediction'][i] = 1

        print(f'--{i}\n{instance_prompt}\n')
        output = open(f'data/tst_gpt3.5-turbo_least2most_prompt{run}_full.txt', 'a+')
        output.write(f'--{i}\n{instance_prompt}\n')
        output.close()

tst_data_df['prediction'].to_csv(f'data/tst_gpt3.5-turbo_least2most_prompt{run}.csv', index=False)

In [ ]:
tst_data_df['prediction'] = 0
func = chatgpt
run = 3
# need to control three variables: question_lists, exemplars, instance

for r in ['xReact', 'oReact', 'xAttr', 'xIntent', 'xNeed']:
    e = 'typing' if r in ['xReact', 'oReact', 'xAttr'] else 'temporal'
    arguments = [c for instance in exemplars[r] for c in instance]
    constant_prompt = least2most_head_prompt + exemplar_prompt(e, question_lists).format(*arguments)
    # print(constant_prompt); continue

    for i in tst_data_df.index[tst_data_df['relation'] == r]:
        tail = tst_data_df['tail'][i]
        head = tst_data_df['head'][i]
        assertion = tst_data_df['assertion'][i]
        if e == 'typing':
            instance_prompt = question_lists[e][0].format(assertion, tail)
        else:
            instance_prompt = question_lists[e][0].format(assertion, tail, head, tail, head)

        # stage 1
        answer = func(constant_prompt + instance_prompt, 5)
        instance_prompt += answer + question_lists[e][1]

        # stage 2
        answer = func(constant_prompt + instance_prompt, 2)
        instance_prompt += answer
        if 'no' in answer.lower():
            print(f'--{i}\n{instance_prompt}\n')
            output = open(f'data/tst_gpt3.5-turbo_least2most_prompt{run}_full.txt', 'a+')
            output.write(f'--{i}\n{instance_prompt}\n')
            output.close()
            continue

        # stage 3
        instance_prompt += question_lists[e][2].format(assertion)
        answer = func(constant_prompt + instance_prompt, 10)
        instance_prompt += answer
        if 'yes' in answer.lower():
            tst_data_df['prediction'][i] = 1

        print(f'--{i}\n{instance_prompt}\n')
        output = open(f'data/tst_gpt3.5-turbo_least2most_prompt{run}_full.txt', 'a+')
        output.write(f'--{i}\n{instance_prompt}\n')
        output.close()

tst_data_df['prediction'].to_csv(f'data/tst_gpt3.5-turbo_least2most_prompt{run}.csv', index=False)

least to most no exemplar

In [ ]:
tst_data_df['prediction'] = 0
func = chatgpt
run = '1_no_exemplar'
# need to control three variables: question_lists_tq, exemplars_tq, assertion_tq

for r in ['xReact', 'oReact', 'xAttr', 'xIntent', 'xNeed']:
    e = 'typing' if r in ['xReact', 'oReact', 'xAttr'] else 'temporal'
    arguments = [c for instance in exemplars_tq[r] for c in instance]
    constant_prompt = least2most_head_prompt #+ exemplar_prompt(e, question_lists_tq).format(*arguments)
    # print(constant_prompt); continue

    for i in tst_data_df.index[tst_data_df['relation'] == r]:
        tail = tst_data_df['tail'][i]
        head = tst_data_df['head'][i]
        assertion_tq = tst_data_df['assertion_tq'][i]
        if e == 'typing':
            instance_prompt = question_lists_tq[e][0].format(assertion_tq, tail)
        else:
            instance_prompt = question_lists_tq[e][0].format(assertion_tq, tail, head, tail, head)

        # stage 1
        answer = func(constant_prompt + instance_prompt, 5)
        instance_prompt += answer + question_lists_tq[e][1]

        # stage 2
        answer = func(constant_prompt + instance_prompt, 2)
        instance_prompt += answer
        if 'no' in answer.lower():
            print(f'--{i}\n{instance_prompt}\n')
            output = open(f'data/tst_gpt3.5-turbo_least2most_prompt{run}_full.txt', 'a+')
            output.write(f'--{i}\n{instance_prompt}\n')
            output.close()
            continue

        # stage 3
        instance_prompt += question_lists_tq[e][2].format(assertion_tq)
        answer = func(constant_prompt + instance_prompt, 10)
        instance_prompt += answer
        if 'yes' in answer.lower():
            tst_data_df['prediction'][i] = 1

        print(f'--{i}\n{instance_prompt}\n')
        output = open(f'data/tst_gpt3.5-turbo_least2most_prompt{run}_full.txt', 'a+')
        output.write(f'--{i}\n{instance_prompt}\n')
        output.close()

tst_data_df['prediction'].to_csv(f'data/tst_gpt3.5-turbo_least2most_prompt{run}.csv', index=False)

In [ ]:
tst_data_df['prediction'] = 0
func = chatgpt
run = '2_no_exemplar'
# need to control three variables: question_lists_tq, exemplars, assertion

for r in ['xReact', 'oReact', 'xAttr', 'xIntent', 'xNeed']:
    e = 'typing' if r in ['xReact', 'oReact', 'xAttr'] else 'temporal'
    arguments = [c for instance in exemplars[r] for c in instance]
    constant_prompt = least2most_head_prompt #+ exemplar_prompt(e, question_lists_tq).format(*arguments)
    # print(constant_prompt); continue

    for i in tst_data_df.index[tst_data_df['relation'] == r]:
        tail = tst_data_df['tail'][i]
        head = tst_data_df['head'][i]
        assertion = tst_data_df['assertion'][i]
        if e == 'typing':
            instance_prompt = question_lists_tq[e][0].format(assertion, tail)
        else:
            instance_prompt = question_lists_tq[e][0].format(assertion, tail, head, tail, head)

        # stage 1
        answer = func(constant_prompt + instance_prompt, 5)
        instance_prompt += answer + question_lists_tq[e][1]

        # stage 2
        answer = func(constant_prompt + instance_prompt, 2)
        instance_prompt += answer
        if 'no' in answer.lower():
            print(f'--{i}\n{instance_prompt}\n')
            output = open(f'data/tst_gpt3.5-turbo_least2most_prompt{run}_full.txt', 'a+')
            output.write(f'--{i}\n{instance_prompt}\n')
            output.close()
            continue

        # stage 3
        instance_prompt += question_lists_tq[e][2].format(assertion)
        answer = func(constant_prompt + instance_prompt, 10)
        instance_prompt += answer
        if 'yes' in answer.lower():
            tst_data_df['prediction'][i] = 1

        print(f'--{i}\n{instance_prompt}\n')
        output = open(f'data/tst_gpt3.5-turbo_least2most_prompt{run}_full.txt', 'a+')
        output.write(f'--{i}\n{instance_prompt}\n')
        output.close()

tst_data_df['prediction'].to_csv(f'data/tst_gpt3.5-turbo_least2most_prompt{run}.csv', index=False)

In [ ]:
tst_data_df['prediction'] = 0
func = chatgpt
run = '3_no_exemplar'
# need to control three variables: question_lists, exemplars, instance

for r in ['xReact', 'oReact', 'xAttr', 'xIntent', 'xNeed']:
    e = 'typing' if r in ['xReact', 'oReact', 'xAttr'] else 'temporal'
    arguments = [c for instance in exemplars[r] for c in instance]
    constant_prompt = least2most_head_prompt #+ exemplar_prompt(e, question_lists).format(*arguments)
    # print(constant_prompt); continue

    for i in tst_data_df.index[tst_data_df['relation'] == r]:
        tail = tst_data_df['tail'][i]
        head = tst_data_df['head'][i]
        assertion = tst_data_df['assertion'][i]
        if e == 'typing':
            instance_prompt = question_lists[e][0].format(assertion, tail)
        else:
            instance_prompt = question_lists[e][0].format(assertion, tail, head, tail, head)

        # stage 1
        answer = func(constant_prompt + instance_prompt, 5)
        instance_prompt += answer + question_lists[e][1]

        # stage 2
        answer = func(constant_prompt + instance_prompt, 2)
        instance_prompt += answer
        if 'no' in answer.lower():
            print(f'--{i}\n{instance_prompt}\n')
            output = open(f'data/tst_gpt3.5-turbo_least2most_prompt{run}_full.txt', 'a+')
            output.write(f'--{i}\n{instance_prompt}\n')
            output.close()
            continue

        # stage 3
        instance_prompt += question_lists[e][2].format(assertion)
        answer = func(constant_prompt + instance_prompt, 10)
        instance_prompt += answer
        if 'yes' in answer.lower():
            tst_data_df['prediction'][i] = 1

        print(f'--{i}\n{instance_prompt}\n')
        output = open(f'data/tst_gpt3.5-turbo_least2most_prompt{run}_full.txt', 'a+')
        output.write(f'--{i}\n{instance_prompt}\n')
        output.close()

tst_data_df['prediction'].to_csv(f'data/tst_gpt3.5-turbo_least2most_prompt{run}.csv', index=False)

random exemplar

In [ ]:
random_5shot_prompt1 = lambda s:judge_prompt_0.format('\n\n'.join(
    ['Statement: {}\nAnswer: {}'.format(tst_random_exemplars[f'{_}_tq'][s], 'Yes' if tst_random_exemplars[f'{_}_tq_label'][s] else 'No') for _ in range(5)] +
    ['Statement: {}\nAnswer: {}'.format(tst_data_df['assertion_tq'][s], '')]
))
print(random_5shot_prompt1(0))

await chatgpt_loop(random_5shot_prompt1, max_tokens=2, filename_suffix='random_5shot_prompt1', resume_at_step=0)

In [ ]:
random_5shot_prompt2 = lambda s:judge_prompt_0.format('\n\n'.join(
    ['Statement: {}\nAnswer: {}'.format(tst_random_exemplars[f'{_}'][s], 'Yes' if tst_random_exemplars[f'{_}_label'][s] else 'No') for _ in range(5)] +
    ['Statement: {}\nAnswer: {}'.format(tst_data_df['assertion'][s], '')]
))
print(random_5shot_prompt2(0))

await chatgpt_loop(random_5shot_prompt2, max_tokens=2, filename_suffix='random_5shot_prompt2', resume_at_step=0)

In [ ]:
random_5shot_prompt3 = lambda s:judge_prompt_1.format('\n\n'.join(
    ['Statement: {}\nAnswer: {}'.format(tst_random_exemplars[f'{_}'][s], 'True' if tst_random_exemplars[f'{_}_label'][s] else 'False') for _ in range(5, 10)] +
    ['Statement: {}\nAnswer: {}'.format(tst_data_df['assertion'][s], '')]
))
print(random_5shot_prompt3(0))

# await chatgpt_loop(random_5shot_prompt3, max_tokens=2, filename_suffix='random_5shot_prompt3', resume_at_step=0)

kate top 5 (regardless label, regardless relation)

more similar to tst instance, the closer to the instance we place it. already checked https://pytorch.org/docs/stable/generated/torch.topk.html

In [ ]:
kate_5shot_prompt1 = lambda s:judge_prompt_0.format('\n\n'.join(
    ['Statement: {}\nAnswer: {}'.format(tst_kate_exemplars_tq[f'{_}_tq'][s], 'Yes' if tst_kate_exemplars_tq[f'{_}_tq_label'][s] else 'No') for _ in [4,3,2,1,0]] +
    ['Statement: {}\nAnswer: {}'.format(tst_data_df['assertion_tq'][s], '')]
))
print(kate_5shot_prompt1(0))

# await chatgpt_loop(kate_5shot_prompt1, max_tokens=2, filename_suffix='kate_5shot_prompt1', resume_at_step=0)

Answer whether the following statement is plausible. Answer with only Yes or No:
Statement: If PersonX push PersonY back, as a result, PersonY or others will, PeopleX step back from PersonX
Answer: Yes

Statement: If PersonX regain PersonY 's composure, can be hindered by, PersonY be disown personx
Answer: Yes

Statement: If PersonX be nowhere, can be hindered by, PersonX friend will not keep PersonY
Answer: Yes

Statement: If PersonX chase PersonZ away, as a result, PersonX will, PersonY lose friend
Answer: Yes

Statement: If PersonX wave PersonY away, as a result, PersonX will, PersonY roll PersonZ eye
Answer: Yes

Statement: If PersonX argue with PersonY, as a result, PersonY or others will, PersonY be not a PersonZ
Answer: 


In [ ]:
kate_5shot_prompt2 = lambda s:judge_prompt_0.format('\n\n'.join(
    ['Statement: {}\nAnswer: {}'.format(tst_kate_exemplars[f'{_}'][s], 'Yes' if tst_kate_exemplars[f'{_}_label'][s] else 'No') for _ in [4,3,2,1,0]] +
    ['Statement: {}\nAnswer: {}'.format(tst_data_df['assertion'][s], '')]
))
print(kate_5shot_prompt2(0))

# await chatgpt_loop(kate_5shot_prompt2, max_tokens=2, filename_suffix='kate_5shot_prompt2', resume_at_step=0)

Answer whether the following statement is plausible. Answer with only Yes or No:
Statement: PersonX stay away from PersonY, thus as an result, PersonX call out to PersonX
Answer: No

Statement: PersonX help the PersonY, thus as an result, PersonX be rebuff by PersonY
Answer: Yes

Statement: PersonX turn down that, thus it can be seen about PersonX's attribute that PersonX get PersonY into trouble
Answer: No

Statement: PersonX be real, thus as an result, PersonY argue PersonZ about it
Answer: Yes

Statement: PersonX challenge PersonZ 's friend, thus, PersonY want PersonY not let
Answer: Yes

Statement: PersonX argue with PersonY, thus as an result, PersonY be not a PersonZ
Answer: 


In [ ]:
kate_5shot_prompt3 = lambda s:judge_prompt_1.format('\n\n'.join(
    ['Statement: {}\nAnswer: {}'.format(tst_kate_exemplars[f'{_}'][s], 'True' if tst_kate_exemplars[f'{_}_label'][s] else 'False') for _ in [4,3,2,1,0]] +
    ['Statement: {}\nAnswer: {}'.format(tst_data_df['assertion'][s], '')]
))
print(kate_5shot_prompt3(0))

# await chatgpt_loop(kate_5shot_prompt3, max_tokens=2, filename_suffix='kate_5shot_prompt3', resume_at_step=0)

Judge the following statement if it's likely to occur, only answer 'True' or 'False':
Statement: PersonX stay away from PersonY, thus as an result, PersonX call out to PersonX
Answer: False

Statement: PersonX help the PersonY, thus as an result, PersonX be rebuff by PersonY
Answer: True

Statement: PersonX turn down that, thus it can be seen about PersonX's attribute that PersonX get PersonY into trouble
Answer: False

Statement: PersonX be real, thus as an result, PersonY argue PersonZ about it
Answer: True

Statement: PersonX challenge PersonZ 's friend, thus, PersonY want PersonY not let
Answer: True

Statement: PersonX argue with PersonY, thus as an result, PersonY be not a PersonZ
Answer: 


kate top 5, exemplars from same relation (regardless label)

In [ ]:
kate_5shot_same_relation_prompt1 = lambda s:judge_prompt_0.format('\n\n'.join(
    ['Statement: {}\nAnswer: {}'.format(tst_kate_same_relation_exemplars_tq[f'{_}_tq'][s], 'Yes' if tst_kate_same_relation_exemplars_tq[f'{_}_tq_label'][s] else 'No') for _ in [4,3,2,1,0]] +
    ['Statement: {}\nAnswer: {}'.format(tst_data_df['assertion_tq'][s], '')]
))
print(kate_5shot_same_relation_prompt1(0))

await chatgpt_loop(kate_5shot_same_relation_prompt1, max_tokens=2, filename_suffix='kate_5shot_same_relation_prompt1', resume_at_step=0)

In [ ]:
kate_5shot_same_relation_prompt2 = lambda s:judge_prompt_0.format('\n\n'.join(
    ['Statement: {}\nAnswer: {}'.format(tst_kate_same_relation_exemplars[f'{_}'][s], 'Yes' if tst_kate_same_relation_exemplars[f'{_}_label'][s] else 'No') for _ in [4,3,2,1,0]] +
    ['Statement: {}\nAnswer: {}'.format(tst_data_df['assertion'][s], '')]
))
print(kate_5shot_same_relation_prompt2(0))

await chatgpt_loop(kate_5shot_same_relation_prompt2, max_tokens=2, filename_suffix='kate_5shot_same_relation_prompt2', resume_at_step=0)

In [ ]:
kate_5shot_same_relation_prompt3 = lambda s:judge_prompt_1.format('\n\n'.join(
    ['Statement: {}\nAnswer: {}'.format(tst_kate_same_relation_exemplars[f'{_}'][s], 'True' if tst_kate_same_relation_exemplars[f'{_}_label'][s] else 'False') for _ in [4,3,2,1,0]] +
    ['Statement: {}\nAnswer: {}'.format(tst_data_df['assertion'][s], '')]
))
print(kate_5shot_same_relation_prompt3(0))

await chatgpt_loop(kate_5shot_same_relation_prompt3, max_tokens=2, filename_suffix='kate_5shot_same_relation_prompt3', resume_at_step=0)

### text-davinci-003

'naive' was run on local runtime

moe ~ system1

In [ ]:
temp_df = tst_data_df[tst_data_df['relation'].isin('xReact,oReact,xAttr'.split(','))]
temp_df.reset_index(inplace=True)
total = (len(temp_df)-1)//bs+1
temp_df

for i in range(0, total):
    while True:
        try:
            print(i, '/', total)
            p = openai.Completion.create(
                model="text-davinci-003",
                prompt=[moe_q1_prompt2.format(temp_df['tail'][s])
                    for s in range(i*bs, min((i+1)*bs, len(temp_df)))
                ],
                max_tokens=5,
                temperature=0
            )
            temp = '\n'.join([x['text'].replace('\n','--') for x in p['choices']])
            print(temp)
            output = open('data/tst_davinci_moe_q1_prompt2.txt', 'a+')
            output.write(f'{i}/{total}\n{temp}\n')
            output.close()
            predictions.extend([x['text'].replace('\n','--') for x in p['choices']])
            time.sleep(timeout_succeed)
            break
        except Exception as e:
            print(e)
            time.sleep(timeout_failure)

In [ ]:
temp_df = tst_data_df[tst_data_df['relation'].isin('xIntent,xNeed'.split(','))]
temp_df.reset_index(inplace=True)
total = (len(temp_df)-1)//bs+1
temp_df

for i in range(0, total):
    while True:
        try:
            print(i, '/', total)
            p = openai.Completion.create(
                model="text-davinci-003",
                prompt=[moe_q3_prompt1.format(temp_df['tail'][s], temp_df['head'][s])
                    for s in range(i*bs, min((i+1)*bs, len(temp_df)))
                ],
                max_tokens=5,
                temperature=0
            )
            temp = '\n'.join([x['text'].replace('\n','--') for x in p['choices']])
            print(temp)
            output = open('data/tst_davinci_moe_q3_prompt1.txt', 'a+')
            output.write(f'{i}/{total}\n{temp}\n')
            output.close()
            predictions.extend([x['text'].replace('\n','--') for x in p['choices']])
            time.sleep(timeout_succeed)
            break
        except Exception as e:
            print(e)
            time.sleep(timeout_failure)

In [ ]:
for i in range(0, total):
    while True:
        try:
            print(i, '/', total)
            p = openai.Completion.create(
                model="text-davinci-003",
                prompt=[moe_q1_prompt1.format(tst_data_df['tail'][s])
                    for s in range(i*bs, min((i+1)*bs, len(tst_data_df)))
                ],
                max_tokens=5,
                temperature=0
            )
            temp = '\n'.join([x['text'].replace('\n','--') for x in p['choices']])
            print(temp)
            output = open('data/tst_davinci_moe_q1_prompt1.txt', 'a+')
            output.write(f'{i}/{total}\n{temp}\n')
            output.close()
            predictions.extend([x['text'].replace('\n','--') for x in p['choices']])
            time.sleep(timeout_succeed)
            break
        except Exception as e:
            print(e)
            time.sleep(timeout_failure)

In [ ]:
for i in range(0, total):
    while True:
        try:
            print(i, '/', total)
            p = openai.Completion.create(
                model="text-davinci-003",
                prompt=[moe_q3_prompt3.format(tst_data_df['tail'][s], tst_data_df['head'][s])
                    for s in range(i*bs, min((i+1)*bs, len(tst_data_df)))
                ],
                max_tokens=5,
                temperature=0
            )
            temp = '\n'.join([x['text'].replace('\n','--') for x in p['choices']])
            print(temp)
            output = open('data/tst_davinci_moe_q3_prompt3.txt', 'a+')
            output.write(f'{i}/{total}\n{temp}\n')
            output.close()
            predictions.extend([x['text'].replace('\n','--') for x in p['choices']])
            time.sleep(timeout_succeed)
            break
        except Exception as e:
            print(e)
            time.sleep(timeout_failure)

zeroshotCoT

In [ ]:
for i in range(0, total):
    while True:
        try:
            print(i, '/', total)
            p = openai.Completion.create(
                model="text-davinci-003",
                prompt=[zeroshotCoT_prompt_tq.format(tst_data_df['assertion'][s])
                    for s in range(i*bs, min((i+1)*bs, len(tst_data_df)))
                ],
                max_tokens=50,
                temperature=0
            )
            temp = '\n'.join([x['text'].replace('\n','--') for x in p['choices']])
            print(temp)
            output = open('data/tst_davinci_zeroshot_prompt1.txt', 'a+')
            output.write(f'{i}/{total}\n{temp}\n')
            output.close()
            predictions.extend([x['text'].replace('\n','--') for x in p['choices']])
            time.sleep(timeout_succeed)
            break
        except Exception as e:
            print(e)
            time.sleep(timeout_failure)

In [ ]:
for i in range(0, total):
    while True:
        try:
            print(i, '/', total)
            p = openai.Completion.create(
                model="text-davinci-003",
                prompt=[zeroshotCoT_prompt1.format(tst_data_df['assertion_tq'][s])
                    for s in range(i*bs, min((i+1)*bs, len(tst_data_df)))
                ],
                max_tokens=50,
                temperature=0
            )
            temp = '\n'.join([x['text'].replace('\n','--') for x in p['choices']])
            print(temp)
            output = open('data/tst_davinci_zeroshot_prompt2.txt', 'a+')
            output.write(f'{i}/{total}\n{temp}\n')
            output.close()
            predictions.extend([x['text'].replace('\n','--') for x in p['choices']])
            time.sleep(timeout_succeed)
            break
        except Exception as e:
            print(e)
            time.sleep(timeout_failure)

In [ ]:
for i in range(0, total):
    while True:
        try:
            print(i, '/', total)
            p = openai.Completion.create(
                model="text-davinci-003",
                prompt=[zeroshotCoT_prompt1.format(tst_data_df['assertion'][s])
                    for s in range(i*bs, min((i+1)*bs, len(tst_data_df)))
                ],
                max_tokens=50,
                temperature=0
            )
            temp = '\n'.join([x['text'].replace('\n','--') for x in p['choices']])
            print(temp)
            output = open('data/tst_davinci_zeroshot_prompt3.txt', 'a+')
            output.write(f'{i}/{total}\n{temp}\n')
            output.close()
            predictions.extend([x['text'].replace('\n','--') for x in p['choices']])
            time.sleep(timeout_succeed)
            break
        except Exception as e:
            print(e)
            time.sleep(timeout_failure)

KATE

In [ ]:
for i in range(0, total):
    while True:
        try:
            print(i, '/', total)
            p = openai.Completion.create(
                model="text-davinci-003",
                prompt=[kate_prompt_0.format('\n\n'.join([
                            'Statement: {}\nAnswer: {}'.format(tst_kate_exemplars['kate_pos_0_tq'][s], 'Yes'),
                            'Statement: {}\nAnswer: {}'.format(tst_kate_exemplars['kate_neg_0_tq'][s], 'No'),
                            'Statement: {}\nAnswer: {}'.format(tst_data_df['assertion_tq'][s], '')]))
                    for s in range(i*bs, min((i+1)*bs, len(tst_data_df)))
                ],
                max_tokens=3,
                temperature=0
            )
            temp = '\n'.join([x['text'].replace('\n','--') for x in p['choices']])
            print(temp)
            output = open('data/tst_davinci_kate_top1_prompt1.txt', 'a+')
            output.write(f'{i}/{total}\n{temp}\n')
            output.close()
            predictions.extend([x['text'].replace('\n','--') for x in p['choices']])
            time.sleep(timeout_succeed)
            break
        except Exception as e:
            print(e)
            time.sleep(timeout_failure)

In [ ]:
for i in range(0, total):
    while True:
        try:
            print(i, '/', total)
            p = openai.Completion.create(
                model="text-davinci-003",
                prompt=[kate_prompt_0.format('\n\n'.join([
                            'Statement: {}\nAnswer: {}'.format(tst_kate_exemplars['kate_pos_0'][s], 'Yes'),
                            'Statement: {}\nAnswer: {}'.format(tst_kate_exemplars['kate_neg_0'][s], 'No'),
                            'Statement: {}\nAnswer: {}'.format(tst_data_df['assertion'][s], '')]))
                    for s in range(i*bs, min((i+1)*bs, len(tst_data_df)))
                ],
                max_tokens=3,
                temperature=0
            )
            temp = '\n'.join([x['text'].replace('\n','--') for x in p['choices']])
            print(temp)
            output = open('data/tst_davinci_kate_top1_prompt2.txt', 'a+')
            output.write(f'{i}/{total}\n{temp}\n')
            output.close()
            predictions.extend([x['text'].replace('\n','--') for x in p['choices']])
            time.sleep(timeout_succeed)
            break
        except Exception as e:
            print(e)
            time.sleep(timeout_failure)

In [ ]:
for i in range(0, total):
    while True:
        try:
            print(i, '/', total)
            p = openai.Completion.create(
                model="text-davinci-003",
                prompt=[kate_prompt_1.format('\n\n'.join([
                            'Statement: {}\nAnswer: {}'.format(tst_kate_exemplars['kate_pos_0'][s], 'True'),
                            'Statement: {}\nAnswer: {}'.format(tst_kate_exemplars['kate_neg_0'][s], 'False'),
                            'Statement: {}\nAnswer: {}'.format(tst_data_df['assertion'][s], '')]))
                    for s in range(i*bs, min((i+1)*bs, len(tst_data_df)))
                ],
                max_tokens=3,
                temperature=0
            )
            temp = '\n'.join([x['text'].replace('\n','--') for x in p['choices']])
            print(temp)
            output = open('data/tst_davinci_kate_top1_prompt3.txt', 'a+')
            output.write(f'{i}/{total}\n{temp}\n')
            output.close()
            predictions.extend([x['text'].replace('\n','--') for x in p['choices']])
            time.sleep(timeout_succeed)
            break
        except Exception as e:
            print(e)
            time.sleep(timeout_failure)

In [ ]:
kate_top5_prompt1 = lambda s:kate_prompt_0.format('\n\n'.join(
    ['Statement: {}\nAnswer: {}'.format(tst_kate_exemplars[f'kate_{p}_{_}_tq'][s], a) \
        for _ in range(5) for p, a in [('pos', 'Yes'), ('neg', 'No')]] +
    ['Statement: {}\nAnswer: {}'.format(tst_data_df['assertion_tq'][s], '')]
))
print(kate_top5_prompt1(0))

davinci_loop(kate_top5_prompt1, max_tokens=4, filename_suffix='kate_top5_prompt1', resume_at_step=0)

In [ ]:
kate_top5_prompt2 = lambda s:kate_prompt_0.format('\n\n'.join(
    ['Statement: {}\nAnswer: {}'.format(tst_kate_exemplars[f'kate_{p}_{_}'][s], a) \
        for _ in range(5) for p, a in [('pos', 'Yes'), ('neg', 'No')]] +
    ['Statement: {}\nAnswer: {}'.format(tst_data_df['assertion'][s], '')]
))
print(kate_top5_prompt2(0))

davinci_loop(kate_top5_prompt2, max_tokens=2, filename_suffix='kate_top5_prompt2', resume_at_step=0)

In [ ]:
kate_top5_prompt3 = lambda s:kate_prompt_1.format('\n\n'.join(
    ['Statement: {}\nAnswer: {}'.format(tst_kate_exemplars[f'kate_{p}_{_}'][s], a) \
        for _ in range(5) for p, a in [('pos', 'True'), ('neg', 'False')]] +
    ['Statement: {}\nAnswer: {}'.format(tst_data_df['assertion'][s], '')]
))
print(kate_top5_prompt3(0))

davinci_loop(kate_top5_prompt3, max_tokens=2, filename_suffix='kate_top5_prompt3', resume_at_step=0)

least2most

In [ ]:
tst_data_df['prediction'] = 0
func = davinci
run = 1
# need to control three variables: question_lists_tq, exemplars_tq, assertion_tq

for r in ['xReact', 'oReact', 'xAttr', 'xIntent', 'xNeed']:
    e = 'typing' if r in ['xReact', 'oReact', 'xAttr'] else 'temporal'
    arguments = [c for instance in exemplars_tq[r] for c in instance]
    constant_prompt = least2most_head_prompt + exemplar_prompt(e, question_lists_tq).format(*arguments)
    # print(constant_prompt); continue

    for i in tst_data_df.index[tst_data_df['relation'] == r]:
        tail = tst_data_df['tail'][i]
        head = tst_data_df['head'][i]
        assertion_tq = tst_data_df['assertion_tq'][i]
        if e == 'typing':
            instance_prompt = question_lists_tq[e][0].format(assertion_tq, tail)
        else:
            instance_prompt = question_lists_tq[e][0].format(assertion_tq, tail, head, tail, head)

        # stage 1
        answer = func(constant_prompt + instance_prompt, 5)
        instance_prompt += answer + question_lists_tq[e][1]

        # stage 2
        answer = func(constant_prompt + instance_prompt, 2)
        instance_prompt += answer
        if 'no' in answer.lower():
            print(f'--{i}\n{instance_prompt}\n')
            output = open(f'data/tst_davinci_least2most_prompt{run}_full.txt', 'a+')
            output.write(f'--{i}\n{instance_prompt}\n')
            output.close()
            continue

        # stage 3
        instance_prompt += question_lists_tq[e][2].format(assertion_tq)
        answer = func(constant_prompt + instance_prompt, 10)
        instance_prompt += answer
        if 'yes' in answer.lower():
            tst_data_df['prediction'][i] = 1

        print(f'--{i}\n{instance_prompt}\n')
        output = open(f'data/tst_davinci_least2most_prompt{run}_full.txt', 'a+')
        output.write(f'--{i}\n{instance_prompt}\n')
        output.close()

tst_data_df['prediction'].to_csv(f'data/tst_davinci_least2most_prompt{run}.csv', index=False)

In [ ]:
tst_data_df['prediction'] = 0
func = davinci
run = 2
# need to control three variables: question_lists_tq, exemplars, assertion

for r in ['xReact', 'oReact', 'xAttr', 'xIntent', 'xNeed']:
    e = 'typing' if r in ['xReact', 'oReact', 'xAttr'] else 'temporal'
    arguments = [c for instance in exemplars[r] for c in instance]
    constant_prompt = least2most_head_prompt + exemplar_prompt(e, question_lists_tq).format(*arguments)
    # print(constant_prompt); continue

    for i in tst_data_df.index[tst_data_df['relation'] == r]:
        tail = tst_data_df['tail'][i]
        head = tst_data_df['head'][i]
        assertion = tst_data_df['assertion'][i]
        if e == 'typing':
            instance_prompt = question_lists_tq[e][0].format(assertion, tail)
        else:
            instance_prompt = question_lists_tq[e][0].format(assertion, tail, head, tail, head)

        # stage 1
        answer = func(constant_prompt + instance_prompt, 5)
        instance_prompt += answer + question_lists_tq[e][1]

        # stage 2
        answer = func(constant_prompt + instance_prompt, 2)
        instance_prompt += answer
        if 'no' in answer.lower():
            print(f'--{i}\n{instance_prompt}\n')
            output = open(f'data/tst_davinci_least2most_prompt{run}_full.txt', 'a+')
            output.write(f'--{i}\n{instance_prompt}\n')
            output.close()
            continue

        # stage 3
        instance_prompt += question_lists_tq[e][2].format(assertion)
        answer = func(constant_prompt + instance_prompt, 10)
        instance_prompt += answer
        if 'yes' in answer.lower():
            tst_data_df['prediction'][i] = 1

        print(f'--{i}\n{instance_prompt}\n')
        output = open(f'data/tst_davinci_least2most_prompt{run}_full.txt', 'a+')
        output.write(f'--{i}\n{instance_prompt}\n')
        output.close()

tst_data_df['prediction'].to_csv(f'data/tst_davinci_least2most_prompt{run}.csv', index=False)

In [ ]:
tst_data_df['prediction'] = 0
func = davinci
run = 3
# need to control three variables: question_lists, exemplars, assertion

for r in ['xReact', 'oReact', 'xAttr', 'xIntent', 'xNeed']:
    e = 'typing' if r in ['xReact', 'oReact', 'xAttr'] else 'temporal'
    arguments = [c for instance in exemplars[r] for c in instance]
    constant_prompt = least2most_head_prompt + exemplar_prompt(e, question_lists).format(*arguments)
    # print(constant_prompt); continue

    for i in tst_data_df.index[tst_data_df['relation'] == r]:
        tail = tst_data_df['tail'][i]
        head = tst_data_df['head'][i]
        assertion = tst_data_df['assertion'][i]
        if e == 'typing':
            instance_prompt = question_lists[e][0].format(assertion, tail)
        else:
            instance_prompt = question_lists[e][0].format(assertion, tail, head, tail, head)

        # stage 1
        answer = func(constant_prompt + instance_prompt, 5)
        instance_prompt += answer + question_lists[e][1]

        # stage 2
        answer = func(constant_prompt + instance_prompt, 2)
        instance_prompt += answer
        if 'no' in answer.lower():
            print(f'--{i}\n{instance_prompt}\n')
            output = open(f'data/tst_davinci_least2most_prompt{run}_full.txt', 'a+')
            output.write(f'--{i}\n{instance_prompt}\n')
            output.close()
            continue

        # stage 3
        instance_prompt += question_lists[e][2].format(assertion)
        answer = func(constant_prompt + instance_prompt, 10)
        instance_prompt += answer
        if 'yes' in answer.lower():
            tst_data_df['prediction'][i] = 1

        print(f'--{i}\n{instance_prompt}\n')
        output = open(f'data/tst_davinci_least2most_prompt{run}_full.txt', 'a+')
        output.write(f'--{i}\n{instance_prompt}\n')
        output.close()

tst_data_df['prediction'].to_csv(f'data/tst_davinci_least2most_prompt{run}.csv', index=False)

least2most no exemplar

In [ ]:
tst_data_df['prediction'] = 0
func = davinci
run = '1_no_exemplar'
# need to control three variables: question_lists_tq, exemplars_tq, assertion_tq

for r in ['xReact', 'oReact', 'xAttr', 'xIntent', 'xNeed']:
    e = 'typing' if r in ['xReact', 'oReact', 'xAttr'] else 'temporal'
    arguments = [c for instance in exemplars_tq[r] for c in instance]
    constant_prompt = least2most_head_prompt #+ exemplar_prompt(e, question_lists_tq).format(*arguments)
    # print(constant_prompt); continue

    for i in tst_data_df.index[tst_data_df['relation'] == r]:
        tail = tst_data_df['tail'][i]
        head = tst_data_df['head'][i]
        assertion_tq = tst_data_df['assertion_tq'][i]
        if e == 'typing':
            instance_prompt = question_lists_tq[e][0].format(assertion_tq, tail)
        else:
            instance_prompt = question_lists_tq[e][0].format(assertion_tq, tail, head, tail, head)

        # stage 1
        answer = func(constant_prompt + instance_prompt, 5)
        instance_prompt += answer + question_lists_tq[e][1]

        # stage 2
        answer = func(constant_prompt + instance_prompt, 2)
        instance_prompt += answer
        if 'no' in answer.lower():
            print(f'--{i}\n{instance_prompt}\n')
            output = open(f'data/tst_davinci_least2most_prompt{run}_full.txt', 'a+')
            output.write(f'--{i}\n{instance_prompt}\n')
            output.close()
            continue

        # stage 3
        instance_prompt += question_lists_tq[e][2].format(assertion_tq)
        answer = func(constant_prompt + instance_prompt, 10)
        instance_prompt += answer
        if 'yes' in answer.lower():
            tst_data_df['prediction'][i] = 1

        print(f'--{i}\n{instance_prompt}\n')
        output = open(f'data/tst_davinci_least2most_prompt{run}_full.txt', 'a+')
        output.write(f'--{i}\n{instance_prompt}\n')
        output.close()

tst_data_df['prediction'].to_csv(f'data/tst_davinci_least2most_prompt{run}.csv', index=False)

In [ ]:
tst_data_df['prediction'] = 0
func = davinci
run = '2_no_exemplar'
# need to control three variables: question_lists_tq, exemplars, assertion

for r in ['xReact', 'oReact', 'xAttr', 'xIntent', 'xNeed']:
    e = 'typing' if r in ['xReact', 'oReact', 'xAttr'] else 'temporal'
    arguments = [c for instance in exemplars[r] for c in instance]
    constant_prompt = least2most_head_prompt #+ exemplar_prompt(e, question_lists_tq).format(*arguments)
    # print(constant_prompt); continue

    for i in tst_data_df.index[tst_data_df['relation'] == r]:
        tail = tst_data_df['tail'][i]
        head = tst_data_df['head'][i]
        assertion = tst_data_df['assertion'][i]
        if e == 'typing':
            instance_prompt = question_lists_tq[e][0].format(assertion, tail)
        else:
            instance_prompt = question_lists_tq[e][0].format(assertion, tail, head, tail, head)

        # stage 1
        answer = func(constant_prompt + instance_prompt, 5)
        instance_prompt += answer + question_lists_tq[e][1]

        # stage 2
        answer = func(constant_prompt + instance_prompt, 2)
        instance_prompt += answer
        if 'no' in answer.lower():
            print(f'--{i}\n{instance_prompt}\n')
            output = open(f'data/tst_davinci_least2most_prompt{run}_full.txt', 'a+')
            output.write(f'--{i}\n{instance_prompt}\n')
            output.close()
            continue

        # stage 3
        instance_prompt += question_lists_tq[e][2].format(assertion)
        answer = func(constant_prompt + instance_prompt, 10)
        instance_prompt += answer
        if 'yes' in answer.lower():
            tst_data_df['prediction'][i] = 1

        print(f'--{i}\n{instance_prompt}\n')
        output = open(f'data/tst_davinci_least2most_prompt{run}_full.txt', 'a+')
        output.write(f'--{i}\n{instance_prompt}\n')
        output.close()

tst_data_df['prediction'].to_csv(f'data/tst_davinci_least2most_prompt{run}.csv', index=False)

In [ ]:
tst_data_df['prediction'] = 0
func = davinci
run = '3_no_exemplar'
# need to control three variables: question_lists, exemplars, assertion

for r in ['xReact', 'oReact', 'xAttr', 'xIntent', 'xNeed']:
    e = 'typing' if r in ['xReact', 'oReact', 'xAttr'] else 'temporal'
    arguments = [c for instance in exemplars[r] for c in instance]
    constant_prompt = least2most_head_prompt #+ exemplar_prompt(e, question_lists).format(*arguments)
    # print(constant_prompt); continue

    for i in tst_data_df.index[tst_data_df['relation'] == r]:
        tail = tst_data_df['tail'][i]
        head = tst_data_df['head'][i]
        assertion = tst_data_df['assertion'][i]
        if e == 'typing':
            instance_prompt = question_lists[e][0].format(assertion, tail)
        else:
            instance_prompt = question_lists[e][0].format(assertion, tail, head, tail, head)

        # stage 1
        answer = func(constant_prompt + instance_prompt, 5)
        instance_prompt += answer + question_lists[e][1]

        # stage 2
        answer = func(constant_prompt + instance_prompt, 2)
        instance_prompt += answer
        if 'no' in answer.lower():
            print(f'--{i}\n{instance_prompt}\n')
            output = open(f'data/tst_davinci_least2most_prompt{run}_full.txt', 'a+')
            output.write(f'--{i}\n{instance_prompt}\n')
            output.close()
            continue

        # stage 3
        instance_prompt += question_lists[e][2].format(assertion)
        answer = func(constant_prompt + instance_prompt, 10)
        instance_prompt += answer
        if 'yes' in answer.lower():
            tst_data_df['prediction'][i] = 1

        print(f'--{i}\n{instance_prompt}\n')
        output = open(f'data/tst_davinci_least2most_prompt{run}_full.txt', 'a+')
        output.write(f'--{i}\n{instance_prompt}\n')
        output.close()

tst_data_df['prediction'].to_csv(f'data/tst_davinci_least2most_prompt{run}.csv', index=False)

re-run

In [ ]:
davinci_loop(random_5shot_prompt1, max_tokens=2, filename_suffix='random_5shot_prompt1', resume_at_step=0)


In [ ]:
davinci_loop(random_5shot_prompt2, max_tokens=2, filename_suffix='random_5shot_prompt2', resume_at_step=0)


In [ ]:
davinci_loop(random_5shot_prompt3, max_tokens=2, filename_suffix='random_5shot_prompt3', resume_at_step=0)


In [ ]:
davinci_loop(kate_5shot_prompt1, max_tokens=2, filename_suffix='kate_5shot_prompt1', resume_at_step=0)


In [ ]:
davinci_loop(kate_5shot_prompt2, max_tokens=2, filename_suffix='kate_5shot_prompt2', resume_at_step=0)


In [ ]:
davinci_loop(kate_5shot_prompt3, max_tokens=2, filename_suffix='kate_5shot_prompt3', resume_at_step=0)


In [ ]:
davinci_loop(kate_5shot_same_relation_prompt1, max_tokens=2, filename_suffix='kate_5shot_same_relation_prompt1', resume_at_step=0)


In [ ]:
davinci_loop(kate_5shot_same_relation_prompt2, max_tokens=2, filename_suffix='kate_5shot_same_relation_prompt2', resume_at_step=0)


In [ ]:
davinci_loop(kate_5shot_same_relation_prompt3, max_tokens=2, filename_suffix='kate_5shot_same_relation_prompt3', resume_at_step=0)


In [ ]:
davinci_loop(naive_prompt1, max_tokens=6, filename_suffix='Jun17-naive_prompt1', resume_at_step=0)


In [ ]:
davinci_loop(naive_prompt2, max_tokens=6, filename_suffix='Jun17-naive_prompt2', resume_at_step=0)


In [ ]:
davinci_loop(naive_prompt3, max_tokens=6, filename_suffix='Jun17-naive_prompt3', resume_at_step=0)


## Generalizability test ~ Different benchmark

Run with prompt design 3 first, then if have time, run with other prompt designs

In [ ]:
tst_data_df = pd.read_csv('data/atomic2020_w_negsample.csv')

tst_kate_exemplars = pd.read_csv('data/ato_kate_exemplars_top5.csv')
# tst_kate_exemplars_tq = pd.read_csv('data/tst_kate_exemplars_tq_top5.csv')
tst_kate_same_relation_exemplars = pd.read_csv('data/ato_kate_same_relation_exemplars_top5.csv')
# tst_kate_same_relation_exemplars_tq = pd.read_csv('data/tst_kate_same_relation_exemplars_tq_top5.csv')
tst_random_exemplars = pd.read_csv('data/ato_random_5exemplars.csv')

# print(tst_data_df.columns, len(tst_kate_exemplars))

In [ ]:
for df in [tst_kate_exemplars,
  # tst_kate_exemplars_tq,
  tst_kate_same_relation_exemplars,
  # tst_kate_same_relation_exemplars_tq,
  tst_random_exemplars]:
  print(df.columns)

Index(['0', '0_label', '1', '1_label', '2', '2_label', '3', '3_label', '4',
       '4_label'],
      dtype='object')
Index(['0', '0_label', '1', '1_label', '2', '2_label', '3', '3_label', '4',
       '4_label'],
      dtype='object')
Index(['0', '0_label', '1', '1_label', '2', '2_label', '3', '3_label', '4',
       '4_label', '0_tq', '0_tq_label', '1_tq', '1_tq_label', '2_tq',
       '2_tq_label', '3_tq', '3_tq_label', '4_tq', '4_tq_label', '5',
       '5_label', '6', '6_label', '7', '7_label', '8', '8_label', '9',
       '9_label'],
      dtype='object')


In [ ]:
predictions = []
# output = open('data/output.txt', 'a+')
# output.close()
# openai.api_key =

In [ ]:
bs = 5 #@param
total = (len(tst_data_df)-1)//bs+1
timeout_succeed = 1 #@param
timeout_failure = 10 #@param

In [ ]:
async def chatgpt_loop_atomic(customize_prompt, max_tokens, filename_suffix, resume_at_step=0, end_at_step=total):
    for i in range(resume_at_step, end_at_step):
        while True:
            try:
                print(i, '/', total)
                pred = await dispatch_openai_requests(
                        messages_list = [[{"role": "user", "content": customize_prompt(s)}]
                            for s in range(i*bs, min((i+1)*bs, len(tst_data_df)))],
                        max_tokens=max_tokens,
                    )
                temp = '\n----------'.join([x['choices'][0]['message']['content'] for x in pred])
                print(temp)
                output = open(f'data/atomic_gpt3.5-turbo_{filename_suffix}.txt', 'a+')
                output.write(f'{i}/{total}\n{temp}\n')
                output.close()
                predictions.extend([x['choices'][0]['message']['content'] for x in pred])
                time.sleep(timeout_succeed)
                break
            except Exception as e:
                print(e)
                time.sleep(timeout_failure)


In [ ]:
def davinci_loop_atomic(customize_prompt, max_tokens, filename_suffix, resume_at_step=0, end_at_step=total):
    for i in range(resume_at_step, end_at_step):
        while True:
            try:
                print(i, '/', total)
                p = openai.Completion.create(
                    model="gpt-3.5-turbo-instruct",
                    prompt=[customize_prompt(s)
                        for s in range(i*bs, min((i+1)*bs, len(tst_data_df)))],
                    max_tokens=max_tokens,
                    temperature=0
                )
                temp = '\n----------'.join([x['text'].replace('\n','--') for x in p['choices']])
                print(temp)
                output = open(f'data/atomic_davinci_{filename_suffix}.txt', 'a+')
                output.write(f'{i}/{total}\n{temp}\n')
                output.close()
                predictions.extend([x['text'].replace('\n','--') for x in p['choices']])
                time.sleep(timeout_succeed)
                break
            except Exception as e:
                print(e)
                time.sleep(timeout_failure)

In [ ]:
cot_examplars_3 = """Q: Judge the following statement if it's likely to occur: After PersonX find PersonY, PersonX wake up on ground.
A: Let's think step by step. If PersonX just wake up on the ground, that means PersonX was sleeping before that. Thus, PersonX cannot be conscious to find another person. Thus, the statement is not likely to occur.

Q: Judge the following statement if it's likely to occur: PeopleX deserve happiness, thus as a result on PersonX's emotion, PersonX reach out to PeopleX.
A: Let's think step by step. The events 'PeopleX deserve happiness' and 'PersonX reach out to PeopleX' are likely irrevelant. Also, the clause 'PersonX reach out to PeopleX' does not describe from PersonX's emotion. Thus, the statement is not likely to occur.

Q: Judge the following statement if it's likely to occur: The event PersonX have a sheet will not happen unless PersonX meet PersonY requirement.
A: Let's think step by step. If PersonX doesn't meet PersonY requirement, PersonX likely doesn't get the reward from PersonY. However, in this case, it's not clear whether 'a sheet' refer to PersonY's reward or not. Thus, the statement is not likely to occur.

Q: Judge the following statement if it's likely to occur: PersonX occupy PersonY position, thus, PersonX want PersonY want to aid in position.
A: Let's think step by step. When PersonX occupy PersonY position, it means PersonY already worked at this position and has experience to do the job. Therefore, it's likely that PersonX want PersonY to aid PerosonX when PersonX is in that job position. Thus, the statement is likely to occur.

Q: Judge the following statement if it's likely to occur: PersonX see that, thus as an result, PersonX want a pet.
A: Let's think step by step. In this context, we can refer the word 'that' as some activity where people play with their pet. Therefore, it stimulates PersonX's desire to have a pet. Thus, the statement is likely to occur.

Q: Judge the following statement if it's likely to occur: {}.
A: """

### gpt3.5-turbo

naive

In [ ]:
judge_prompt_1 = "Judge the following statement if it's likely to occur, only answer 'True' or 'False':\n{}"
naive_prompt3 = lambda s:judge_prompt_1.format(tst_data_df['assertion'][s])
print(naive_prompt3(0))

await chatgpt_loop_atomic(naive_prompt3, max_tokens=2, filename_suffix='naive_prompt3', resume_at_step=0)

typing constraint

In [ ]:
idx_q1 = tst_data_df['relation'].isin(['xReact', 'oReact', 'xAttr'])
temp_df = tst_data_df[idx_q1]
temp_df.reset_index(inplace=True)
total = (len(temp_df)-1)//bs+1
# temp_df

In [ ]:
i = 3
prompt = moe_q1_prompt1.format(temp_df['tail'][i])
print(prompt)
chatgpt(prompt, max_tokens=3)

Which aspect (among three options 1. event/activity, 2. persona, 3. mental state) of the subject does the clause 'PersonX feel more rested for the next day' express. Answer the choice only.


'3. mental'

In [ ]:
# run MoE first
for i in range(0, total):
    while True:
        try:
            print(i, '/', total)
            pred = await dispatch_openai_requests(
                    messages_list = [[{"role": "user", "content": moe_q1_prompt1.format(s)}] for s in temp_df['tail'][i*bs:(i+1)*bs]],
                    max_tokens=4,
                )
            temp = ', '.join([x['choices'][0]['message']['content'] for x in pred])
            print(temp)
            output = open('data/atomic_gpt3.5-turbo_moe_q1_prompt1.txt', 'a+')
            output.write(f'{i}/{total}\n{temp}\n')
            output.close()
            predictions.extend([x['choices'][0]['message']['content'] for x in pred])
            time.sleep(timeout_succeed)
            break
        except Exception as e:
            print(e)
            time.sleep(timeout_failure)


temporal constraint

In [ ]:
idx_q3 = tst_data_df['relation'].isin(['xIntent', 'xNeed'])
temp_df = tst_data_df[idx_q3]
temp_df.reset_index(inplace=True)
total = (len(temp_df)-1)//bs+1
# temp_df

In [ ]:
i = 0
prompt = moe_q3_prompt3.format(temp_df['tail'][i], temp_df['head'][i])
print(prompt)
chatgpt(prompt, max_tokens=3)

Which one of the following two statements is more plausible:
0. PersonX swim in the water before PersonX pulls ___ out of the water
1. PersonX swim in the water after PersonX pulls ___ out of the water
Answer 0 or 1 only.


'0'

In [ ]:
for b in range(0, total):
    while True:
        try:
            print(b, '/', total)
            pred = await dispatch_openai_requests(
                    messages_list = [[{"role": "user", "content": moe_q3_prompt3.format(temp_df['tail'][i], temp_df['head'][i])}] \
                        for i in range(b*bs, min((b+1)*bs, len(temp_df)))],
                    max_tokens=4,
                )
            temp = ', '.join([x['choices'][0]['message']['content'] for x in pred])
            print(temp)
            output = open('data/atomic_gpt3.5-turbo_moe_q3_prompt3.txt', 'a+')
            output.write(f'{b}/{total}\n{temp}\n')
            output.close()
            predictions.extend([x['choices'][0]['message']['content'] for x in pred])
            time.sleep(timeout_succeed)
            break
        except Exception as e:
            print(e)
            time.sleep(timeout_failure)


zeroshotCoT

In [ ]:
zeroshot_prompt3 = lambda s:zeroshotCoT_prompt1.format(tst_data_df['assertion'][s])
print(zeroshot_prompt3(0))

await chatgpt_loop_atomic(zeroshot_prompt3, max_tokens=100, filename_suffix='zeroshot_prompt3', resume_at_step=0)

random exemplar

In [ ]:
random_5shot_prompt3 = lambda s:judge_prompt_1.format('\n\n'.join(
    ['Statement: {}\nAnswer: {}'.format(tst_random_exemplars[f'{_}'][s], 'True' if tst_random_exemplars[f'{_}_label'][s] else 'False') for _ in range(5, 10)] +
    ['Statement: {}\nAnswer: {}'.format(tst_data_df['assertion'][s], '')]
))
print(random_5shot_prompt3(0))

await chatgpt_loop_atomic(random_5shot_prompt3, max_tokens=2, filename_suffix='random_5shot_prompt3', resume_at_step=0)

kate top 5 (regardless label, regardless relation)

more similar to tst instance, the closer to the instance we place it. already checked https://pytorch.org/docs/stable/generated/torch.topk.html

In [ ]:
kate_5shot_prompt3 = lambda s:judge_prompt_1.format('\n\n'.join(
    ['Statement: {}\nAnswer: {}'.format(tst_kate_exemplars[f'{_}'][s], 'True' if tst_kate_exemplars[f'{_}_label'][s] else 'False') for _ in [4,3,2,1,0]] +
    ['Statement: {}\nAnswer: {}'.format(tst_data_df['assertion'][s], '')]
))
print(kate_5shot_prompt3(0))

await chatgpt_loop_atomic(kate_5shot_prompt3, max_tokens=2, filename_suffix='kate_5shot_prompt3', resume_at_step=0)

kate top 5, exemplars from same relation (regardless label)

In [ ]:
kate_5shot_same_relation_prompt3 = lambda s:judge_prompt_1.format('\n\n'.join(
    ['Statement: {}\nAnswer: {}'.format(tst_kate_same_relation_exemplars[f'{_}'][s], 'True' if tst_kate_same_relation_exemplars[f'{_}_label'][s] else 'False') for _ in [4,3,2,1,0]] +
    ['Statement: {}\nAnswer: {}'.format(tst_data_df['assertion'][s], '')]
))
print(kate_5shot_same_relation_prompt3(0))

await chatgpt_loop_atomic(kate_5shot_same_relation_prompt3, max_tokens=2, filename_suffix='kate_5shot_same_relation_prompt3', resume_at_step=0)

In [ ]:
randomcot_prompt3 = lambda s:cot_examplars_3.format(tst_data_df['assertion'][s])
print(randomcot_prompt3(0))

await chatgpt_loop_atomic(randomcot_prompt3, max_tokens=80, filename_suffix='randomcot_prompt3', resume_at_step=0)

### text-davinci-003

In [ ]:
naive_prompt3 = lambda s:judge_prompt_1.format(tst_data_df['assertion'][s])
print(naive_prompt3(0))

davinci_loop_atomic(naive_prompt3, max_tokens=10, filename_suffix='naive_prompt3', resume_at_step=0)

In [ ]:
naive_prompt2 = lambda s:judge_prompt_0.format(tst_data_df['assertion'][s])
print(naive_prompt2(0))

davinci_loop_atomic(naive_prompt2, max_tokens=10, filename_suffix='naive_prompt2', resume_at_step=3)

typing constraint

In [ ]:
temp_df = tst_data_df[tst_data_df['relation'].isin('xReact,oReact,xAttr'.split(','))]
temp_df.reset_index(inplace=True)
total = (len(temp_df)-1)//bs+1
temp_df

In [ ]:
i = 0
prompt = moe_q1_prompt1.format(temp_df['tail'][i])
print(prompt)
davinci(prompt, max_tokens=3)

In [ ]:
for i in range(0, total):
    while True:
        try:
            print(i, '/', total)
            p = openai.Completion.create(
                model="text-davinci-003",
                prompt=[moe_q1_prompt1.format(temp_df['tail'][s])
                    for s in range(i*bs, min((i+1)*bs, len(temp_df)))
                ],
                max_tokens=5,
                temperature=0
            )
            temp = '\n----------'.join([x['text'].replace('\n','--') for x in p['choices']])
            print(temp)
            output = open('data/atomic_davinci_moe_q1_prompt1.txt', 'a+')
            output.write(f'{i}/{total}\n{temp}\n')
            output.close()
            predictions.extend([x['text'].replace('\n','--') for x in p['choices']])
            time.sleep(timeout_succeed)
            break
        except Exception as e:
            print(e)
            time.sleep(timeout_failure)

temporal constraint

In [ ]:
temp_df = tst_data_df[tst_data_df['relation'].isin('xIntent,xNeed'.split(','))]
temp_df.reset_index(inplace=True)
total = (len(temp_df)-1)//bs+1
temp_df

,index,head,relation,tail,label,modification,assertion
0,2,PersonX pulls ___ out of the water,xNeed,PersonX swim in the water,1,0,The event PersonX pulls ___ out of the water w...
1,3,PersonX makes PersonY smile,xIntent,PersonX feel loved,0,2,"PersonX makes PersonY smile, thus it can be se..."
2,7,PersonX does n't want to waste it,xIntent,PersonX make a budget,0,2,"PersonX does n't want to waste it, thus it can..."
3,8,PersonX evaluates the patient 's ___,xIntent,PersonX intended to take care of the patient,1,0,"PersonX evaluates the patient 's ___, thus it ..."
4,10,PersonX throws the ball,xIntent,PersonX intended to play catch,1,0,"PersonX throws the ball, thus it can be seen a..."
...,...,...,...,...,...,...,...
559,992,PersonX watches PersonY,xNeed,PersonX be watchful,0,1,The event PersonX watches PersonY will not hap...
560,994,PersonX represents every ___,xNeed,PersonX be well known,1,0,The event PersonX represents every ___ will no...
561,995,PersonX finds ___ to describe,xIntent,PersonX feel less worried,0,2,"PersonX finds ___ to describe, thus it can be ..."
562,997,PersonX is making a cake,xNeed,PersonX feel happy,0,2,The event PersonX is making a cake will not ha...


In [ ]:
i = 2
prompt = moe_q3_prompt3.format(temp_df['tail'][i], temp_df['head'][i])
print(prompt)
davinci(prompt, max_tokens=3)

Which one of the following two statements is more plausible:
0. PersonX make a budget before PersonX does n't want to waste it
1. PersonX make a budget after PersonX does n't want to waste it
Answer 0 or 1 only.


'0'

In [ ]:
for i in range(0, total):
    while True:
        try:
            print(i, '/', total)
            p = openai.Completion.create(
                model="text-davinci-003",
                prompt=[moe_q3_prompt3.format(temp_df['tail'][s], temp_df['head'][s])
                    for s in range(i*bs, min((i+1)*bs, len(temp_df)))
                ],
                max_tokens=5,
                temperature=0
            )
            temp = '\n----------'.join([x['text'].replace('\n','--') for x in p['choices']])
            print(temp)
            output = open('data/atomic_davinci_moe_q3_prompt3.txt', 'a+')
            output.write(f'{i}/{total}\n{temp}\n')
            output.close()
            predictions.extend([x['text'].replace('\n','--') for x in p['choices']])
            time.sleep(timeout_succeed)
            break
        except Exception as e:
            print(e)
            time.sleep(timeout_failure)

other baselines

In [ ]:
zeroshot_prompt3 = lambda s:zeroshotCoT_prompt1.format(tst_data_df['assertion'][s])
for i in range(1):
  print(zeroshot_prompt3(i))

davinci_loop_atomic(zeroshot_prompt3, max_tokens=100, filename_suffix='zeroshot_prompt3', resume_at_step=1, end_at_step=200)

In [ ]:
zeroshot_prompt3 = lambda s:zeroshotCoT_prompt1.format(tst_data_df['assertion'][s])
for i in range(1):
  print(zeroshot_prompt3(i))

davinci(zeroshot_prompt3(0))

Judge the statement 'PersonX pries open, thus it can be seen about PersonX's attribute that PersonX tools' if it's likely to occur. Let's think step by step, then conclude by answering 'True' or 'False'.


'False'

In [ ]:
for i in range(1, 5):
  print(zeroshot_prompt3(i))
  print(davinci(zeroshot_prompt3(i), max_tokens=100))

Judge the statement 'PersonX sees a snake, thus it can be seen about PersonX's attribute that PersonX be frightened' if it's likely to occur. Let's think step by step, then conclude by answering 'True' or 'False'.
True
Judge the statement 'The event PersonX pulls ___ out of the water will not happen unless PersonX swim in the water' if it's likely to occur. Let's think step by step, then conclude by answering 'True' or 'False'.
True
Judge the statement 'PersonX makes PersonY smile, thus it can be seen about PersonX's intention that PersonX feel loved' if it's likely to occur. Let's think step by step, then conclude by answering 'True' or 'False'.
Step 1: PersonX makes PersonY smile.
- This statement is likely to occur, as it is a simple action that can easily be done by PersonX.

Step 2: It can be seen about PersonX's intention that PersonX feels loved.
- This statement is not necessarily true. While making someone smile can be a sign of affection and love, it does not necessarily mean

In [ ]:
random_5shot_prompt3 = lambda s:judge_prompt_1.format('\n\n'.join(
    ['Statement: {}\nAnswer: {}'.format(tst_random_exemplars[f'{_}'][s], 'True' if tst_random_exemplars[f'{_}_label'][s] else 'False') for _ in range(5, 10)] +
    ['Statement: {}\nAnswer: {}'.format(tst_data_df['assertion'][s], '')]
))
print(random_5shot_prompt3(0))

davinci_loop_atomic(random_5shot_prompt3, max_tokens=2, filename_suffix='random_5shot_prompt3', resume_at_step=0)

In [ ]:
kate_5shot_prompt3 = lambda s:judge_prompt_1.format('\n\n'.join(
    ['Statement: {}\nAnswer: {}'.format(tst_kate_exemplars[f'{_}'][s], 'True' if tst_kate_exemplars[f'{_}_label'][s] else 'False') for _ in [4,3,2,1,0]] +
    ['Statement: {}\nAnswer: {}'.format(tst_data_df['assertion'][s], '')]
))
print(kate_5shot_prompt3(0))

davinci_loop_atomic(kate_5shot_prompt3, max_tokens=2, filename_suffix='kate_5shot_prompt3', resume_at_step=0)

In [ ]:
kate_5shot_same_relation_prompt3 = lambda s:judge_prompt_1.format('\n\n'.join(
    ['Statement: {}\nAnswer: {}'.format(tst_kate_same_relation_exemplars[f'{_}'][s], 'True' if tst_kate_same_relation_exemplars[f'{_}_label'][s] else 'False') for _ in [4,3,2,1,0]] +
    ['Statement: {}\nAnswer: {}'.format(tst_data_df['assertion'][s], '')]
))
print(kate_5shot_same_relation_prompt3(0))

davinci_loop_atomic(kate_5shot_same_relation_prompt3, max_tokens=2, filename_suffix='kate_5shot_same_relation_prompt3', resume_at_step=0)

In [ ]:
randomcot_prompt3 = lambda s:cot_examplars_3.format(tst_data_df['assertion'][s])
print(randomcot_prompt3(0))

davinci_loop_atomic(randomcot_prompt3, max_tokens=80, filename_suffix='randomcot_prompt3', resume_at_step=0)

## Summary of baselines

In [ ]:
fout = open('data/output_kate_davinci_June9.txt', 'a+')
fout.write('\n'.join([str(x) for x in predictions]))
fout.close()

In [ ]:
# naive
naive1 = "Answer whether the following statement is plausible. Answer with only Yes or No:\n{}".format(tst_data_df['assertion_tq'])
naive2 = "Answer whether the following statement is plausible. Answer with only Yes or No:\n{}".format(tst_data_df['assertion'])
naive3 = "Judge the following statement if it's likely to occur, only answer 'True' or 'False':\n{}".format(tst_data_df['assertion'])

In [ ]:
# naive + kate(-sr)

In [ ]:
# zeroshot CoT.
zeroshotCoT_prompt_tq = "Answer whether the statement '{}' is plausible. Let's think step by step, then conclude by answering 'True' or 'False'."
zeroshotCoT_prompt1 = "Judge the statement '{}' if it's likely to occur. Let's think step by step, then conclude by answering 'True' or 'False'." # if add only, it will strictly follow that command, only produce 'True' or 'False' w/o CoT

# the combination zeroshotCoT_prompt_tq + assertion_tq is so baddd!
zeroshot_prompt1 = zeroshotCoT_prompt_tq.format(assertion)
zeroshot_prompt2 = zeroshotCoT_prompt1.format(assertion_tq)
zeroshot_prompt3 = zeroshotCoT_prompt1.format(assertion)

In [ ]:
# activeCoT
# automateCoT

In [ ]:
# least2most. this setting will get the final answer directly (not as plug-and-play as system1)
# please refer to the first two cells in 'Least to Most' subsection

In [ ]:
# system1
# serve as an add-on. Only run once as deterministic.

# typing
moe_q1_prompt1 = "Which aspect (among three options 1. event/activity, 2. persona, 3. mental state) of the subject does the clause '{}' express. Answer the choice only."

# temporal
moe_q3_prompt3 = """Which one of the following two statements is more plausible:
0. {0} before {1}
1. {0} after {1}
Answer 0 or 1 only.""" # tail [] head, 1 -> after, 0 -> before

# ablation ~ different prompt choice for typing and temporal constraint
moe_q1_prompt2 = "Determine if clause '{}' expresses an event or activity of the subject. Answer 'Yes' or 'No' only."
moe_q3_prompt1 = "Judge if the event '{}' likely occurs after the event '{}'. Answer 'Yes' or 'No' only."    # pretty unstable

# in the result table in Overleaf
# (baselineX + moe) = avg(baselineX i + moe for i from 1 to 3)

In [ ]:
# activeCoT, pool of exemplar
{
    "prompt": [
        {
            "question": "Q: Judge the following statement if it's likely to occur, only answer 'True' or 'False': 'PersonX learns to play the trumpet, thus, PersonY wants to applaud PersonX's talent.'\nA:",
            "rationale": "",
            "pred_ans": "True"
        },
        {
            "question": "Q: Judge the following statement if it's likely to occur, only answer 'True' or 'False': 'PersonX eat most of those, thus as a result on PersonX's emotion, PersonX feel hungry.'\nA:",
            "rationale": "",
            "pred_ans": "False"
        },
        {
            "question": "Q: Judge the following statement if it's likely to occur, only answer 'True' or 'False': 'PersonX do not feel well, thus as a result on PersonX's emotion, PersonX only make it through half.'\nA:",
            "rationale": "",
            "pred_ans": "False"
        },
        {
            "question": "Q: Judge the following statement if it's likely to occur, only answer 'True' or 'False': 'Before PersonX really empathize with PersonY, PersonX feel.'\nA:",
            "rationale": "",
            "pred_ans": "False"
        },
        {
            "question": "Q: Judge the following statement if it's likely to occur, only answer 'True' or 'False': 'PersonX have a traumatic incident, thus it can be seen about PersonX's attribute that PersonX be introvert'\nA:",
            "rationale": "",
            "pred_ans": "False"
        }

    ]
}

## End